In [1]:
import findspark
findspark.init()
findspark.find()

# Normal Imports
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, TimestampType 

from pyspark.sql.functions import *
from pyspark.sql.window import Window

print("imported")

location = "/Users/dinesh/Desktop/DINESH H R/Programming/Data Engineering/Course  Contents/scenarios_data/"

imported


In [2]:
# spark context
conf = SparkConf().setAppName("Linkedin").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf = conf)
sc.setLogLevel("ERROR")

# spark session
spark = SparkSession.builder.master("local[1]").appName('Linkedin scenario').getOrCreate()

print("imported")

23/12/20 16:45:31 WARN Utils: Your hostname, DINESHs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.15 instead (on interface en0)
23/12/20 16:45:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/20 16:45:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/20 16:45:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


imported


In [3]:
data = [{"id": 1, "Fruit": 'Apple'},
{"id": 2, "Fruit": 'Bananna'}]

spark.createDataFrame(data,schema= ['fruit','id']).show()

+-------+---+
|  fruit| id|
+-------+---+
|  Apple|  1|
|Bananna|  2|
+-------+---+



In [4]:
data = [("Alice", 25), ("Bob", 30), ("Alice", 25),("katy",32)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)

# Drop duplicates based on 'Name' and 'Age'
deduplicated_df = df.dropDuplicates(subset=["Name", "Age"])
deduplicated_df.show()
# 2. Using df.distinct()

df.distinct().show()

+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
| katy| 32|
+-----+---+

+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
| katy| 32|
+-----+---+



In [5]:
data = [("Hello world and hello PySpark",),
    ("This is a PySpark example",),
    ("Let's count unique words",)]

schema = ["text"]
df = spark.createDataFrame(data, schema)
df.show(truncate=False)

# Split the text column into words and explode to create one row per word
words_df = df.withColumn("word", explode(split(df["text"], " ")))
words_df.show(truncate=False)


+-----------------------------+
|text                         |
+-----------------------------+
|Hello world and hello PySpark|
|This is a PySpark example    |
|Let's count unique words     |
+-----------------------------+

+-----------------------------+-------+
|text                         |word   |
+-----------------------------+-------+
|Hello world and hello PySpark|Hello  |
|Hello world and hello PySpark|world  |
|Hello world and hello PySpark|and    |
|Hello world and hello PySpark|hello  |
|Hello world and hello PySpark|PySpark|
|This is a PySpark example    |This   |
|This is a PySpark example    |is     |
|This is a PySpark example    |a      |
|This is a PySpark example    |PySpark|
|This is a PySpark example    |example|
|Let's count unique words     |Let's  |
|Let's count unique words     |count  |
|Let's count unique words     |unique |
|Let's count unique words     |words  |
+-----------------------------+-------+



In [6]:
# Calculate the total number of unique words

words_df.select(countDistinct("word")).show()

unique_word_count = words_df.select(countDistinct("word")).collect()[0][0]
print(unique_word_count)

# Show the total number of unique words
print("Total number of unique words:", unique_word_count)

+--------------------+
|count(DISTINCT word)|
+--------------------+
|                  13|
+--------------------+

13
Total number of unique words: 13


1. https://www.linkedin.com/feed/update/urn:li:activity:7101496064259604480/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7101496064259604480%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29


In [7]:
# https://www.linkedin.com/feed/update/urn:li:activity:7101496064259604480/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7101496064259604480%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29 -- 

columns = ["user_id", "timestamp"]

# data= [("user1", "2023-08-21 10:00:00"),
# ("user2", "2023-08-21 11:30:00"),
# ("user1", "2023-08-21 12:15:00"),
# ("user3", "2023-08-21 13:45:00"),
# ("user2", "2023-08-21 14:30:00"),
# ("user1", "2023-08-21 15:00:00")]

data = [
("user1", "2023-08-21 10:09:09"),
("user2", "2023-08-21 11:30:00"),
("user1", "2023-08-21 12:15:00"),
("user3", "2023-08-21 13:45:09"),
("user2", "2023-08-21 14:30:00"),
("user1", "2023-08-21 15:09:09")
]

# Questions:
# 📌 𝐅𝐢𝐧𝐝 𝐭𝐡𝐞 𝐞𝐚𝐫𝐥𝐢𝐞𝐬𝐭 𝐚𝐧𝐝 𝐥𝐚𝐭𝐞𝐬𝐭 𝐭𝐢𝐦𝐞𝐬𝐭𝐚𝐦𝐩𝐬 𝐢𝐧 𝐭𝐡𝐞 𝐝𝐚𝐭𝐚𝐬𝐞𝐭.
# 📌 𝐂𝐨𝐮𝐧𝐭 𝐭𝐡𝐞 𝐧𝐮𝐦𝐛𝐞𝐫 𝐨𝐟 𝐚𝐜𝐭𝐢𝐯𝐢𝐭𝐢𝐞𝐬 𝐩𝐞𝐫 𝐮𝐬𝐞𝐫.
# 📌 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐭𝐢𝐦𝐞 𝐝𝐮𝐫𝐚𝐭𝐢𝐨𝐧 𝐛𝐞𝐭𝐰𝐞𝐞𝐧 𝐜𝐨𝐧𝐬𝐞𝐜𝐮𝐭𝐢𝐯𝐞 𝐚𝐜𝐭𝐢𝐯𝐢𝐭𝐢𝐞𝐬 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐮𝐬𝐞𝐫.

In [8]:
data = spark.createDataFrame(data, schema=columns)
data.withColumn("timestamp1", to_timestamp(data.timestamp))
data.show()
data.printSchema()

+-------+-------------------+
|user_id|          timestamp|
+-------+-------------------+
|  user1|2023-08-21 10:09:09|
|  user2|2023-08-21 11:30:00|
|  user1|2023-08-21 12:15:00|
|  user3|2023-08-21 13:45:09|
|  user2|2023-08-21 14:30:00|
|  user1|2023-08-21 15:09:09|
+-------+-------------------+

root
 |-- user_id: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [9]:
# method 1
# 📌 𝐅𝐢𝐧𝐝 𝐭𝐡𝐞 𝐞𝐚𝐫𝐥𝐢𝐞𝐬𝐭 𝐚𝐧𝐝 𝐥𝐚𝐭𝐞𝐬𝐭 𝐭𝐢𝐦𝐞𝐬𝐭𝐚𝐦𝐩𝐬 𝐢𝐧 𝐭𝐡𝐞 𝐝𝐚𝐭𝐚𝐬𝐞𝐭.
ordered_desc = data.orderBy(desc(data.timestamp))
print("Latest")
ordered_desc.limit(1).show()

ordered_asc = data.orderBy(data.timestamp)
print("Earliest")
ordered_asc.limit(1).show()

Latest
+-------+-------------------+
|user_id|          timestamp|
+-------+-------------------+
|  user1|2023-08-21 15:09:09|
+-------+-------------------+

Earliest
+-------+-------------------+
|user_id|          timestamp|
+-------+-------------------+
|  user1|2023-08-21 10:09:09|
+-------+-------------------+



In [10]:
# method 2
# 📌 𝐅𝐢𝐧𝐝 𝐭𝐡𝐞 𝐞𝐚𝐫𝐥𝐢𝐞𝐬𝐭 𝐚𝐧𝐝 𝐥𝐚𝐭𝐞𝐬𝐭 𝐭𝐢𝐦𝐞𝐬𝐭𝐚𝐦𝐩𝐬 𝐢𝐧 𝐭𝐡𝐞 𝐝𝐚𝐭𝐚𝐬𝐞𝐭.
max_time_stamp = data.withColumn("timestamp", to_timestamp(data.timestamp))
max_time_stamp.show()

max_time = max_time_stamp.selectExpr("max(timestamp)").first()[0]
print(max_time)

min_time = max_time_stamp.selectExpr("min(timestamp)").first()[0]
print(min_time)

+-------+-------------------+
|user_id|          timestamp|
+-------+-------------------+
|  user1|2023-08-21 10:09:09|
|  user2|2023-08-21 11:30:00|
|  user1|2023-08-21 12:15:00|
|  user3|2023-08-21 13:45:09|
|  user2|2023-08-21 14:30:00|
|  user1|2023-08-21 15:09:09|
+-------+-------------------+

2023-08-21 15:09:09
2023-08-21 10:09:09


In [11]:
# 📌 𝐂𝐨𝐮𝐧𝐭 𝐭𝐡𝐞 𝐧𝐮𝐦𝐛𝐞𝐫 𝐨𝐟 𝐚𝐜𝐭𝐢𝐯𝐢𝐭𝐢𝐞𝐬 𝐩𝐞𝐫 𝐮𝐬𝐞𝐫.

count_data = data.groupBy(data.user_id).agg(count("*").alias("activity_count"))
count_data.show()

+-------+--------------+
|user_id|activity_count|
+-------+--------------+
|  user1|             3|
|  user2|             2|
|  user3|             1|
+-------+--------------+



In [12]:
# 📌 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐭𝐢𝐦𝐞 𝐝𝐮𝐫𝐚𝐭𝐢𝐨𝐧 𝐛𝐞𝐭𝐰𝐞𝐞𝐧 𝐜𝐨𝐧𝐬𝐞𝐜𝐮𝐭𝐢𝐯𝐞 𝐚𝐜𝐭𝐢𝐯𝐢𝐭𝐢𝐞𝐬 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐮𝐬𝐞𝐫.

windowspec = Window.partitionBy("user_id").orderBy("timestamp")
windowdata = data.withColumn("next_timestamp", lead("timestamp").over(windowspec))
windowdata.show()

windowdata.printSchema()

+-------+-------------------+-------------------+
|user_id|          timestamp|     next_timestamp|
+-------+-------------------+-------------------+
|  user1|2023-08-21 10:09:09|2023-08-21 12:15:00|
|  user1|2023-08-21 12:15:00|2023-08-21 15:09:09|
|  user1|2023-08-21 15:09:09|               null|
|  user2|2023-08-21 11:30:00|2023-08-21 14:30:00|
|  user2|2023-08-21 14:30:00|               null|
|  user3|2023-08-21 13:45:09|               null|
+-------+-------------------+-------------------+

root
 |-- user_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- next_timestamp: string (nullable = true)



In [13]:
# windowdata.withColumn("difference", minute(col("next_timestamp"))- minute(col("timestamp"))).show()

windowdata.withColumn("difference", col("timestamp").cast("Long")).show()

+-------+-------------------+-------------------+----------+
|user_id|          timestamp|     next_timestamp|difference|
+-------+-------------------+-------------------+----------+
|  user1|2023-08-21 10:09:09|2023-08-21 12:15:00|      null|
|  user1|2023-08-21 12:15:00|2023-08-21 15:09:09|      null|
|  user1|2023-08-21 15:09:09|               null|      null|
|  user2|2023-08-21 11:30:00|2023-08-21 14:30:00|      null|
|  user2|2023-08-21 14:30:00|               null|      null|
|  user3|2023-08-21 13:45:09|               null|      null|
+-------+-------------------+-------------------+----------+



In [14]:
# Linked in code for above code.

schema = ["user_id","timestamp"]
data=[
("user1", "2023-08-21 10:09:09"),
("user2", "2023-08-21 11:30:00"),
("user1", "2023-08-21 12:15:00"),
("user3", "2023-08-21 13:45:09"),
("user2", "2023-08-21 14:30:00"),
("user1", "2023-08-21 15:09:09")]

# Create a Dataframe from the data and schema
df = spark.createDataFrame(data, schema)
df.show()
df.printSchema()

df = df.withColumn("timestamp", to_timestamp(col("timestamp")))
df.show()
df.printSchema()

+-------+-------------------+
|user_id|          timestamp|
+-------+-------------------+
|  user1|2023-08-21 10:09:09|
|  user2|2023-08-21 11:30:00|
|  user1|2023-08-21 12:15:00|
|  user3|2023-08-21 13:45:09|
|  user2|2023-08-21 14:30:00|
|  user1|2023-08-21 15:09:09|
+-------+-------------------+

root
 |-- user_id: string (nullable = true)
 |-- timestamp: string (nullable = true)

+-------+-------------------+
|user_id|          timestamp|
+-------+-------------------+
|  user1|2023-08-21 10:09:09|
|  user2|2023-08-21 11:30:00|
|  user1|2023-08-21 12:15:00|
|  user3|2023-08-21 13:45:09|
|  user2|2023-08-21 14:30:00|
|  user1|2023-08-21 15:09:09|
+-------+-------------------+

root
 |-- user_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [15]:
# Find the earliest and latest timestamps in the dataset.
max_timestamp=df.select(max("timestamp")).first()[0]
print("max timestamp:",max_timestamp)
min_timestamp=df.select(min("timestamp")).first()[0]
print("min_timestamp",min_timestamp)
 
# Count the number of activities per user.
no_of_activities = df.groupBy("user_id").agg(count("*").alias("activity_count"))
no_of_activities.show()

max timestamp: 2023-08-21 15:09:09
min_timestamp 2023-08-21 10:09:09
+-------+--------------+
|user_id|activity_count|
+-------+--------------+
|  user1|             3|
|  user2|             2|
|  user3|             1|
+-------+--------------+



In [16]:
#Calculate the time duration between consecutive activities for each user.
window_spec = Window.partitionBy("user_id").orderBy("timestamp")
df_with_next_timestamp = df.withColumn("next_timestamp", lead("timestamp").over(window_spec))
 
df_with_next_timestamp.show()

+-------+-------------------+-------------------+
|user_id|          timestamp|     next_timestamp|
+-------+-------------------+-------------------+
|  user1|2023-08-21 10:09:09|2023-08-21 12:15:00|
|  user1|2023-08-21 12:15:00|2023-08-21 15:09:09|
|  user1|2023-08-21 15:09:09|               null|
|  user2|2023-08-21 11:30:00|2023-08-21 14:30:00|
|  user2|2023-08-21 14:30:00|               null|
|  user3|2023-08-21 13:45:09|               null|
+-------+-------------------+-------------------+



In [17]:
#Calculate the time duration between consecutive actions
df_with_duration = df_with_next_timestamp.withColumn("duration", (col("next_timestamp").cast("long") - col("timestamp").cast("long")))
# Show the final DataFrame
df_with_duration.show()

+-------+-------------------+-------------------+--------+
|user_id|          timestamp|     next_timestamp|duration|
+-------+-------------------+-------------------+--------+
|  user1|2023-08-21 10:09:09|2023-08-21 12:15:00|    7551|
|  user1|2023-08-21 12:15:00|2023-08-21 15:09:09|   10449|
|  user1|2023-08-21 15:09:09|               null|    null|
|  user2|2023-08-21 11:30:00|2023-08-21 14:30:00|   10800|
|  user2|2023-08-21 14:30:00|               null|    null|
|  user3|2023-08-21 13:45:09|               null|    null|
+-------+-------------------+-------------------+--------+



2. https://www.linkedin.com/feed/update/urn:li:activity:7105024721942933504?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7105024721942933504%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [18]:
# https://www.linkedin.com/feed/update/urn:li:activity:7101175506905022464/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7101175506905022464%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# student_id, subject, score.

# Sample Dataset :

# student_id,subject,score
# 1,Math,85
# 2,Science,92
# 3,Math,78
# 4,English,88
# 5,Science,95
# 6,Math,90

# 🔹 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐚𝐯𝐞𝐫𝐚𝐠𝐞 𝐬𝐜𝐨𝐫𝐞 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐬𝐮𝐛𝐣𝐞𝐜𝐭.
# 🔹 𝐈𝐝𝐞𝐧𝐭𝐢𝐟𝐲 𝐭𝐡𝐞 𝐡𝐢𝐠𝐡𝐞𝐬𝐭 𝐬𝐜𝐨𝐫𝐞 𝐚𝐧𝐝 𝐢𝐭𝐬 𝐜𝐨𝐫𝐫𝐞𝐬𝐩𝐨𝐧𝐝𝐢𝐧𝐠 𝐬𝐭𝐮𝐝𝐞𝐧𝐭 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐬𝐮𝐛𝐣𝐞𝐜𝐭.
# 🔹 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐭𝐨𝐭𝐚𝐥 𝐧𝐮𝐦𝐛𝐞𝐫 𝐨𝐟 𝐬𝐭𝐮𝐝𝐞𝐧𝐭𝐬 𝐰𝐡𝐨 𝐭𝐨𝐨𝐤 𝐞𝐚𝐜𝐡 𝐬𝐮𝐛𝐣𝐞𝐜𝐭.
# 🔹 𝐅𝐢𝐧𝐝 𝐭𝐡𝐞 𝐬𝐮𝐛𝐣𝐞𝐜𝐭(𝐬) 𝐰𝐢𝐭𝐡 𝐭𝐡𝐞 𝐡𝐢𝐠𝐡𝐞𝐬𝐭 𝐚𝐯𝐞𝐫𝐚𝐠𝐞 𝐬𝐜𝐨𝐫𝐞.

In [19]:
schema = ["student_id","subject","score"]
data = [(1,"Math",85),
(2,"Science",92),
(3,"Math",78),
(4,"English",88),
(5,"Science",95),
(6,"Math",90)]

df = spark.createDataFrame(data, schema=schema)
df.show()

+----------+-------+-----+
|student_id|subject|score|
+----------+-------+-----+
|         1|   Math|   85|
|         2|Science|   92|
|         3|   Math|   78|
|         4|English|   88|
|         5|Science|   95|
|         6|   Math|   90|
+----------+-------+-----+



In [20]:
# 🔹 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐚𝐯𝐞𝐫𝐚𝐠𝐞 𝐬𝐜𝐨𝐫𝐞 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐬𝐮𝐛𝐣𝐞𝐜𝐭.
df1 = df.groupBy("subject").agg(avg("score").alias("avg_score"))
df1.show()

+-------+-----------------+
|subject|        avg_score|
+-------+-----------------+
|   Math|84.33333333333333|
|Science|             93.5|
|English|             88.0|
+-------+-----------------+



In [21]:
# 🔹 𝐈𝐝𝐞𝐧𝐭𝐢𝐟𝐲 𝐭𝐡𝐞 𝐡𝐢𝐠𝐡𝐞𝐬𝐭 𝐬𝐜𝐨𝐫𝐞 𝐚𝐧𝐝 𝐢𝐭𝐬 𝐜𝐨𝐫𝐫𝐞𝐬𝐩𝐨𝐧𝐝𝐢𝐧𝐠 𝐬𝐭𝐮𝐝𝐞𝐧𝐭 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐬𝐮𝐛𝐣𝐞𝐜𝐭.
df2 = df.groupBy("subject").agg(max("score").alias("max_score"))
df2.show()

+-------+---------+
|subject|max_score|
+-------+---------+
|   Math|       90|
|Science|       95|
|English|       88|
+-------+---------+



In [22]:
# 🔹 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐭𝐨𝐭𝐚𝐥 𝐧𝐮𝐦𝐛𝐞𝐫 𝐨𝐟 𝐬𝐭𝐮𝐝𝐞𝐧𝐭𝐬 𝐰𝐡𝐨 𝐭𝐨𝐨𝐤 𝐞𝐚𝐜𝐡 𝐬𝐮𝐛𝐣𝐞𝐜𝐭.
df3 = df.groupBy("subject").agg(count("*").alias("total_students"))
df3.show()

+-------+--------------+
|subject|total_students|
+-------+--------------+
|   Math|             3|
|Science|             2|
|English|             1|
+-------+--------------+



In [23]:
# 🔹 𝐅𝐢𝐧𝐝 𝐭𝐡𝐞 𝐬𝐮𝐛𝐣𝐞𝐜𝐭(𝐬) 𝐰𝐢𝐭𝐡 𝐭𝐡𝐞 𝐡𝐢𝐠𝐡𝐞𝐬𝐭 𝐚𝐯𝐞𝐫𝐚𝐠𝐞 𝐬𝐜𝐨𝐫𝐞.
avger = df.selectExpr("avg(score)").collect()[0][0]
print(avger)
df.filter(df.score > avger).show()

88.0
+----------+-------+-----+
|student_id|subject|score|
+----------+-------+-----+
|         2|Science|   92|
|         5|Science|   95|
|         6|   Math|   90|
+----------+-------+-----+



In [24]:
# 🔹 𝐅𝐢𝐧𝐝 𝐭𝐡𝐞 𝐬𝐮𝐛𝐣𝐞𝐜𝐭(𝐬) 𝐰𝐢𝐭𝐡 𝐭𝐡𝐞 𝐡𝐢𝐠𝐡𝐞𝐬𝐭 𝐚𝐯𝐞𝐫𝐚𝐠𝐞 𝐬𝐜𝐨𝐫𝐞.

# df1.orderBy(desc("avg_score")).limit(1).show()
df1.orderBy(col("avg_score").desc()).limit(1).collect()[0][1]
# df1.orderBy(desc("avg_score")).limit(1).collect()[0][1]

93.5

In [25]:
# https://www.linkedin.com/feed/update/urn:li:activity:7099432968850472960/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7099432968850472960%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
# Dataset: The dataset contains the following columns: order_id, customer_id, order_date, total_amount

# order_id,customer_id,order_date,total_amount
# 1,C101,2023-07-01,150
# 2,C102,2023-07-02,200
# 3,C101,2023-07-02,100
# 4,C103,2023-07-03,300
# 5,C102,2023-07-04,250
# 6,C101,2023-07-05,120

# 🔹 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐭𝐨𝐭𝐚𝐥 𝐫𝐞𝐯𝐞𝐧𝐮𝐞 𝐠𝐞𝐧𝐞𝐫𝐚𝐭𝐞𝐝 𝐟𝐫𝐨𝐦 𝐚𝐥𝐥 𝐨𝐫𝐝𝐞𝐫𝐬.
# 🔹 𝐅𝐢𝐧𝐝 𝐭𝐡𝐞 𝐚𝐯𝐞𝐫𝐚𝐠𝐞 𝐨𝐫𝐝𝐞𝐫 𝐚𝐦𝐨𝐮𝐧𝐭.
# 🔹 𝐈𝐝𝐞𝐧𝐭𝐢𝐟𝐲 𝐭𝐡𝐞 𝐡𝐢𝐠𝐡𝐞𝐬𝐭 𝐭𝐨𝐭𝐚𝐥 𝐨𝐫𝐝𝐞𝐫 𝐚𝐦𝐨𝐮𝐧𝐭 𝐚𝐧𝐝 𝐢𝐭𝐬 𝐜𝐨𝐫𝐫𝐞𝐬𝐩𝐨𝐧𝐝𝐢𝐧𝐠 𝐜𝐮𝐬𝐭𝐨𝐦𝐞𝐫.
# 🔹 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐭𝐨𝐭𝐚𝐥 𝐧𝐮𝐦𝐛𝐞𝐫 𝐨𝐟 𝐨𝐫𝐝𝐞𝐫𝐬 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐜𝐮𝐬𝐭𝐨𝐦𝐞𝐫.

In [26]:
schema = ["order_id","customer_id","order_date","total_amount"]

data = [(1,"C101","2023-07-01",150),
(2,"C102","2023-07-02",200),
(3,"C101","2023-07-02",100),
(4,"C103","2023-07-03",300),
(5,"C102","2023-07-04",250),
(6,"C101","2023-07-05",120)]

df = spark.createDataFrame(data, schema=schema)
df.show()

+--------+-----------+----------+------------+
|order_id|customer_id|order_date|total_amount|
+--------+-----------+----------+------------+
|       1|       C101|2023-07-01|         150|
|       2|       C102|2023-07-02|         200|
|       3|       C101|2023-07-02|         100|
|       4|       C103|2023-07-03|         300|
|       5|       C102|2023-07-04|         250|
|       6|       C101|2023-07-05|         120|
+--------+-----------+----------+------------+



In [27]:
# 🔹 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐭𝐨𝐭𝐚𝐥 𝐫𝐞𝐯𝐞𝐧𝐮𝐞 𝐠𝐞𝐧𝐞𝐫𝐚𝐭𝐞𝐝 𝐟𝐫𝐨𝐦 𝐚𝐥𝐥 𝐨𝐫𝐝𝐞𝐫𝐬
df.selectExpr("sum(total_amount) as sum").show()

# 🔹 𝐅𝐢𝐧𝐝 𝐭𝐡𝐞 𝐚𝐯𝐞𝐫𝐚𝐠𝐞 𝐨𝐫𝐝𝐞𝐫 𝐚𝐦𝐨𝐮𝐧𝐭.
df.selectExpr("avg(total_amount) as avg").show()

+----+
| sum|
+----+
|1120|
+----+

+------------------+
|               avg|
+------------------+
|186.66666666666666|
+------------------+



In [28]:
# 🔹 𝐈𝐝𝐞𝐧𝐭𝐢𝐟𝐲 𝐭𝐡𝐞 𝐡𝐢𝐠𝐡𝐞𝐬𝐭 𝐭𝐨𝐭𝐚𝐥 𝐨𝐫𝐝𝐞𝐫 𝐚𝐦𝐨𝐮𝐧𝐭 𝐚𝐧𝐝 𝐢𝐭𝐬 𝐜𝐨𝐫𝐫𝐞𝐬𝐩𝐨𝐧𝐝𝐢𝐧𝐠 𝐜𝐮𝐬𝐭𝐨𝐦𝐞𝐫.
# df.selectExpr("max(total_amount)").show()

df.orderBy(desc("total_amount")).select("customer_id", "total_amount").limit(1).show()

+-----------+------------+
|customer_id|total_amount|
+-----------+------------+
|       C103|         300|
+-----------+------------+



In [29]:
# 🔹 𝐂𝐚𝐥𝐜𝐮𝐥𝐚𝐭𝐞 𝐭𝐡𝐞 𝐭𝐨𝐭𝐚𝐥 𝐧𝐮𝐦𝐛𝐞𝐫 𝐨𝐟 𝐨𝐫𝐝𝐞𝐫𝐬 𝐟𝐨𝐫 𝐞𝐚𝐜𝐡 𝐜𝐮𝐬𝐭𝐨𝐦𝐞𝐫.
df.groupBy("customer_id").agg(count("*").alias("count")).show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|       C101|    3|
|       C102|    2|
|       C103|    1|
+-----------+-----+



In [30]:
# https://www.linkedin.com/feed/update/urn:li:activity:7098861202759450624/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7098861202759450624%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
# Dataset: The dataset contains the following columns: employee_id, employee_name, department, salary.
# Sample Dataset:
# employee_id,employee_name,department,salary
# 1,John Doe,Engineering,90000
# 2,Jane Smith,Marketing,75000
# 3,Michael Johnson,Engineering,105000
# 4,Emily Davis,Marketing,80000
# 5,Robert Brown,Engineering,95000
# 6,Linda Wilson,HR,60000

# Questions:
# ⏺ Calculate the total payroll cost for the company.
# ⏺ Find the average salary for each department.
# ⏺ Identify the highest-paid employee and their department.
# ⏺ Calculate the total number of employees in each department.

In [31]:
schema = ["employee_id","employee_name","department","salary"]
data = [(1,"John Doe","Engineering",90000),
(2,"Jane Smith","Marketing",75000),
(3,"Michael Johnson","Engineering",105000),
(4,"Emily Davis","Marketing",80000),
(5,"Robert Brown","Engineering",95000),
(6,"Linda Wilson","HR",60000)]

df = spark.createDataFrame(data, schema=schema)
df.show()

+-----------+---------------+-----------+------+
|employee_id|  employee_name| department|salary|
+-----------+---------------+-----------+------+
|          1|       John Doe|Engineering| 90000|
|          2|     Jane Smith|  Marketing| 75000|
|          3|Michael Johnson|Engineering|105000|
|          4|    Emily Davis|  Marketing| 80000|
|          5|   Robert Brown|Engineering| 95000|
|          6|   Linda Wilson|         HR| 60000|
+-----------+---------------+-----------+------+



In [32]:
# ⏺ Calculate the total payroll cost for the company.
df.select(sum("salary").alias("sum")).show()
# ⏺ Find the average salary for each department.
df.groupBy("department").agg(avg("salary").alias("avg")).show()

+------+
|   sum|
+------+
|505000|
+------+

+-----------+-----------------+
| department|              avg|
+-----------+-----------------+
|Engineering|96666.66666666667|
|  Marketing|          77500.0|
|         HR|          60000.0|
+-----------+-----------------+



In [33]:
# ⏺ Identify the highest-paid employee and their department.
df.orderBy(desc("salary")).select("employee_name", "department").limit(1).show()

# ⏺ Calculate the total number of employees in each department.
df.groupBy("department").agg(count("*").alias("count")).show()

+---------------+-----------+
|  employee_name| department|
+---------------+-----------+
|Michael Johnson|Engineering|
+---------------+-----------+

+-----------+-----+
| department|count|
+-----------+-----+
|Engineering|    3|
|  Marketing|    2|
|         HR|    1|
+-----------+-----+



In [34]:
# https://www.linkedin.com/feed/update/urn:li:activity:7098006304308297729/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7098006304308297729%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
# The dataset contains the following columns: product_id, product_name, category, price, quantity_sold.

# product_id,product_name,category,price,quantity_sold
# 1,Product A,Electronics,500,100
# 2,Product B,Clothing,50,200
# 3,Product C,Electronics,800,50
# 4,Product D,Beauty,30,300
# 5,Product E,Clothing,75,150

# 🔸 Calculate the total revenue generated from all sales.
# 🔸 Find the top 5 best-selling products based on the quantity sold.
# 🔸 Calculate the average price of products in each category.
# 🔸 Identify the category with the highest total revenue.

In [35]:
schema = ["product_id","product_name","category","price","quantity_sold"]
data = [
(1,"Product A","Electronics",500,100),
(2,"Product B","Clothing",50,200),
(3,"Product C","Electronics",800,50),
(4,"Product D",'Beauty',30,300),
(5,"Product E","Clothing",75,150)]

df = spark.createDataFrame(data, schema=schema)
df.show()

+----------+------------+-----------+-----+-------------+
|product_id|product_name|   category|price|quantity_sold|
+----------+------------+-----------+-----+-------------+
|         1|   Product A|Electronics|  500|          100|
|         2|   Product B|   Clothing|   50|          200|
|         3|   Product C|Electronics|  800|           50|
|         4|   Product D|     Beauty|   30|          300|
|         5|   Product E|   Clothing|   75|          150|
+----------+------------+-----------+-----+-------------+



In [36]:
# 🔸 Calculate the total revenue generated from all sales.

# revenue = df.withColumn("revenue", (df.price + df.quantity_sold)).agg(sum(col("revenue"))).collect()[0][0]
# print(revenue)

revenue1 = df.withColumn("revenue", (df.price + df.quantity_sold))
revenue1.show()
revenue2 = revenue1.selectExpr("sum(revenue)").collect()[0][0]
print(revenue2)

+----------+------------+-----------+-----+-------------+-------+
|product_id|product_name|   category|price|quantity_sold|revenue|
+----------+------------+-----------+-----+-------------+-------+
|         1|   Product A|Electronics|  500|          100|    600|
|         2|   Product B|   Clothing|   50|          200|    250|
|         3|   Product C|Electronics|  800|           50|    850|
|         4|   Product D|     Beauty|   30|          300|    330|
|         5|   Product E|   Clothing|   75|          150|    225|
+----------+------------+-----------+-----+-------------+-------+

2255


In [37]:
# 🔸 Find the top 5 best-selling products based on the quantity sold.
df.sort(desc(df.quantity_sold)).limit(5).show()

# 🔸 Calculate the average price of products in each category.
df.groupBy(df.category).agg(avg(df.price).alias("avg_price")).show()

# 🔸 Identify the category with the highest total revenue.
revenue1.sort(desc(revenue1.revenue)).select("category").limit(1).show()
# print(revenue1.sort(desc(revenue1.revenue)).select("category").first())

+----------+------------+-----------+-----+-------------+
|product_id|product_name|   category|price|quantity_sold|
+----------+------------+-----------+-----+-------------+
|         4|   Product D|     Beauty|   30|          300|
|         2|   Product B|   Clothing|   50|          200|
|         5|   Product E|   Clothing|   75|          150|
|         1|   Product A|Electronics|  500|          100|
|         3|   Product C|Electronics|  800|           50|
+----------+------------+-----------+-----+-------------+

+-----------+---------+
|   category|avg_price|
+-----------+---------+
|Electronics|    650.0|
|   Clothing|     62.5|
|     Beauty|     30.0|
+-----------+---------+

+-----------+
|   category|
+-----------+
|Electronics|
+-----------+



In [38]:
# https://www.linkedin.com/feed/update/urn:li:activity:7097033199851081728/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7097033199851081728%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
# Sample CSV file named customer_purchases.csv with the following columns:

# customer_id,purchase_amount,purchase_date
# 1,100,2023-01-15
# 2,150,2023-02-20
# 1,200,2023-03-10
# 3,50,2023-04-05
# 2,120,2023-05-15
# 1,300,2023-06-25

# Find the customer with the highest total purchase amount.❔
# Practise this question and boost your pyspark skill. 💡💡

In [39]:
schema = ["customer_id","purchase_amount","purchase_date"]
data = [(1,100,"2023-01-15"),
(2,150,"2023-02-20"),
(1,200,"2023-03-10"),
(3,50,"2023-04-05"),
(2,120,"2023-05-15"),
(1,300,"2023-06-25")]

df = spark.createDataFrame(data, schema=schema)
df.show()

+-----------+---------------+-------------+
|customer_id|purchase_amount|purchase_date|
+-----------+---------------+-------------+
|          1|            100|   2023-01-15|
|          2|            150|   2023-02-20|
|          1|            200|   2023-03-10|
|          3|             50|   2023-04-05|
|          2|            120|   2023-05-15|
|          1|            300|   2023-06-25|
+-----------+---------------+-------------+



In [40]:
# Find the customer with the highest total purchase amount.❔
df.groupBy(df.customer_id).agg(max(df.purchase_amount).alias("max_amount")).sort(desc(col("max_amount"))).limit(1).show()

df.orderBy(desc(df.purchase_amount)).limit(1).show()

row_value = df.orderBy(desc(df.purchase_amount)).limit(1)
row_value.show()

+-----------+----------+
|customer_id|max_amount|
+-----------+----------+
|          1|       300|
+-----------+----------+

+-----------+---------------+-------------+
|customer_id|purchase_amount|purchase_date|
+-----------+---------------+-------------+
|          1|            300|   2023-06-25|
+-----------+---------------+-------------+

+-----------+---------------+-------------+
|customer_id|purchase_amount|purchase_date|
+-----------+---------------+-------------+
|          1|            300|   2023-06-25|
+-----------+---------------+-------------+



In [41]:
# https://www.linkedin.com/feed/update/urn:li:activity:7107898245263065088/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7107898245263065088%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
# Question : How can you create a DataFrame?
# You can create a DataFrame from an existing RDD using the createDataFrame method.



In [42]:
# https://www.linkedin.com/feed/update/urn:li:activity:7108381435144671232/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7108381435144671232%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
# Question : handling row duplication in PySpark DataFrames! 📊

# 1. Dropping Duplicates:
# The simplest approach is to drop duplicate rows based on a subset of columns. This can be done using the dropDuplicates() method. Here's how:

data = [("Alice", 25), ("Bob", 30), ("Alice", 25),("katy",32)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)
df.show()

# Drop duplicates based on 'Name' and 'Age'
deduplicated_df = df.dropDuplicates(subset=["Name", "Age"])
deduplicated_df.show()

# 2. Using df.distinct()

df.distinct().show()

# 3. Using Window Functions:
# You can also leverage window functions to assign row numbers and then filter out duplicate rows.

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
window_spec = Window.partitionBy("Name", "Age").orderBy("Name")

# Add a row number column
df_with_row_number = df.withColumn("row_number", row_number().over(window_spec))

# Filter out duplicate rows
deduplicated_windowfn_df = df_with_row_number.filter(df_with_row_number.row_number == 1).drop("row_number")

# Show result
deduplicated_windowfn_df.show()

+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
|Alice| 25|
| katy| 32|
+-----+---+

+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
| katy| 32|
+-----+---+

+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
| katy| 32|
+-----+---+

+-----+---+
| Name|Age|
+-----+---+
|Alice| 25|
|  Bob| 30|
| katy| 32|
+-----+---+



In [43]:
# https://www.linkedin.com/feed/update/urn:li:activity:7108623025624809472/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7108623025624809472%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29



In [44]:
# https://www.linkedin.com/feed/update/urn:li:activity:7108743821303193600/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7108743821303193600%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# PySpark UDF with an example.
# PySpark User-Defined Functions (UDFs) are your secret weapon for custom transformations on DataFrame columns.

from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

data = [("New York", 95.0), ("Los Angeles", 78.5), ("Chicago", 82.0)]
columns = ["City", "Temperature_F"]
df = spark.createDataFrame(data, columns)
df.show()

def fahrenheit_to_celsius(fahrenheit_temp):
  celsius_temp = ((fahrenheit_temp - 32) * 5 / 9)
  return round(celsius_temp, 2)

# Register UDF
convert_to_celsius_udf = udf(fahrenheit_to_celsius, FloatType())

# Apply UDF to create a new column
# df_with_celsius = df.withColumn("Temperature_C", convert_to_celsius_udf(df.Temperature_F))

# df_with_celsius = df.selectExpr("City", "Temperature_F", "((Temperature_F - 32) * 5 / 9) as Temperature_C")
df_with_celsius = df.withColumn("Temperature_C", round(((df.Temperature_F - 32) * 5 / 9), 2))

df_with_celsius.show()

+-----------+-------------+
|       City|Temperature_F|
+-----------+-------------+
|   New York|         95.0|
|Los Angeles|         78.5|
|    Chicago|         82.0|
+-----------+-------------+

+-----------+-------------+-------------+
|       City|Temperature_F|Temperature_C|
+-----------+-------------+-------------+
|   New York|         95.0|         35.0|
|Los Angeles|         78.5|        25.83|
|    Chicago|         82.0|        27.78|
+-----------+-------------+-------------+



In [45]:
# https://www.linkedin.com/feed/update/urn:li:activity:7109430095672479744/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7109430095672479744%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
#

data = [(1, "Alice", "2021-01-15", 100),
  (2, "Bob", "2021-03-20", 200),
  (3, "Charlie", "2021-02-10", 150)]

columns = ["id", "name", "birthdate", "salary"]

df = spark.createDataFrame(data, columns)
df.show()

# 🔹Selecting Columns with Alias:
df.selectExpr("name AS full_name", "salary * 1.1 AS updated_salary").show()

# 🔹Mathematical Transformations:
df.selectExpr("salary", "salary * 1.5 AS increased_salary").show()

# 🔹String Manipulation:
df.selectExpr("name", "substring(birthdate, 1, 4) AS birth_year", "concat(name, ' - ', birth_year) AS name_year").show()

# 🔹Conditional Expressions:
df.selectExpr("name", "CASE WHEN salary > 150 THEN 'High Salary' ELSE 'Low Salary' END AS salary_category").show()

# 🔹Type Casting:
df.selectExpr("name", "cast(salary AS double) AS double_salary").show()


+---+-------+----------+------+
| id|   name| birthdate|salary|
+---+-------+----------+------+
|  1|  Alice|2021-01-15|   100|
|  2|    Bob|2021-03-20|   200|
|  3|Charlie|2021-02-10|   150|
+---+-------+----------+------+

+---------+--------------+
|full_name|updated_salary|
+---------+--------------+
|    Alice|         110.0|
|      Bob|         220.0|
|  Charlie|         165.0|
+---------+--------------+

+------+----------------+
|salary|increased_salary|
+------+----------------+
|   100|           150.0|
|   200|           300.0|
|   150|           225.0|
+------+----------------+

+-------+----------+--------------+
|   name|birth_year|     name_year|
+-------+----------+--------------+
|  Alice|      2021|  Alice - 2021|
|    Bob|      2021|    Bob - 2021|
|Charlie|      2021|Charlie - 2021|
+-------+----------+--------------+

+-------+---------------+
|   name|salary_category|
+-------+---------------+
|  Alice|     Low Salary|
|    Bob|    High Salary|
|Charlie|     Low S

In [46]:
# https://www.linkedin.com/feed/update/urn:li:activity:7099953413865582592/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7099953413865582592%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# The map function in Spark is your go-to tool for transforming your data. It takes an input RDD (Resilient Distributed Dataset) and applies a transformation function to each element, producing a new RDD. Think of it as a way to modify, clean, or reshape your data to fit your analysis needs. Whether you're changing data types, extracting features, or even just performing calculations, map is your Swiss Army knife.

# # Spark Context Setup
# from pyspark import SparkContext
# sc = SparkContext("local", "MapFunctionExample")

# # Sample RDD
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)

# # Applying the Map Transformation
squared_rdd = rdd.map(lambda x: x ** 2)

# # Collecting Results
result = squared_rdd.collect()
print(result) # Output: [1, 4, 9, 16, 25]

# 🟢 ReduceByKey Function: Aggregating with Style
# When it comes to aggregating data, the reduceByKey function is your ace in the hole. It's perfect for performing operations on key-value pairs, where values for the same key are grouped together and combined using a provided function. This can be incredibly useful for tasks like summing values for each key or finding maximum values.

# # Sample Key-Value RDD
data = [("A", 10), ("B", 5), ("A", 20), ("B", 8), ("C", 15)]
rdd = sc.parallelize(data)

# # Applying the ReduceByKey Transformation
sum_by_key = rdd.reduceByKey(lambda a, b: a + b)

# # Collecting Results
result = sum_by_key.collect()
print(result) # Output: [('A', 30), ('B', 13), ('C', 15)]


# 💡 Pro Tip: Remember that both map and reduceByKey functions are part of Spark's resilient distributed processing model, allowing you to efficiently handle large datasets across a cluster of machines.
# So there you have it, the dynamic duo of Spark functions—map for flexible data transformation and reduceByKey for effective aggregation.


[1, 4, 9, 16, 25]
[('B', 13), ('C', 15), ('A', 30)]


In [47]:
# https://www.linkedin.com/feed/update/urn:li:activity:7110200923343261696/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7110200923343261696%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# Create a DataFrame with sales data
data = [(1,10.0, 2.0), (2,8.0, float('nan')), (3,12.0, 3.0), (4,float('nan'), 5.0)]
df = spark.createDataFrame(data, ["product_id","quantity_sold", "quantity_returned"])
df.show()

# Calculate net quantity sold, handling NaN values
net_sales_df = df.withColumn("quantity_sold_withoutNull", nanvl(df["quantity_sold"], lit(0.0)))\
    .withColumn("quantity_returned_withoutNull", nanvl(df["quantity_returned"], lit(0.0)))
    
result_df = net_sales_df.withColumn("net_quantity_sold", \
                        net_sales_df["quantity_sold_withoutNull"] - net_sales_df["quantity_returned_withoutNull"])

# Show the result
result_df.drop('quantity_sold','quantity_returned').show()

+----------+-------------+-----------------+
|product_id|quantity_sold|quantity_returned|
+----------+-------------+-----------------+
|         1|         10.0|              2.0|
|         2|          8.0|              NaN|
|         3|         12.0|              3.0|
|         4|          NaN|              5.0|
+----------+-------------+-----------------+

+----------+-------------------------+-----------------------------+-----------------+
|product_id|quantity_sold_withoutNull|quantity_returned_withoutNull|net_quantity_sold|
+----------+-------------------------+-----------------------------+-----------------+
|         1|                     10.0|                          2.0|              8.0|
|         2|                      8.0|                          0.0|              8.0|
|         3|                     12.0|                          3.0|              9.0|
|         4|                      0.0|                          5.0|             -5.0|
+----------+------------------

In [48]:
# https://www.linkedin.com/feed/update/urn:li:activity:7110228203385884672/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7110228203385884672%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# Write a pyspark query to 
# 1. convert product_name in lowercase without leading or trailing white spaces.
# 2. set sale_date in the format ('YYYY-MM').
# 3. calculate total the number of times the product was sold in this month.
# Return the result table ordered by product_name in ascending order. In case of a tie, order it by sale_date in ascending order.

# The query result format is in the following example.

# Sales
# +---------+--------------+------------+
# | sale_id | product_name | sale_date  |
# +---------+--------------+------------+
# | 1       | LCPHONE      | 2000-01-16 |
# | 2       | LCPhone      | 2000-01-17 |
# | 3       | LcPhOnE      | 2000-02-18 |
# | 4       | LCKeyCHAiN   | 2000-02-19 |
# | 5       | LCKeyChain   | 2000-02-28 |
# | 6       | Matryoshka   | 2000-03-31 |
# +---------+--------------+------------+

# Result table:
# +--------------+-----------+-------+
# | product_name | sale_date | total |
# +--------------+-----------+-------+
# | lckeychain   | 2000-02   | 2     |
# | lcphone      | 2000-01   | 2     |
# | lcphone      | 2000-02   | 1     |
# | matryoshka   | 2000-03   | 1     |
# +--------------+-----------+-------+

# Pyspark Solution:

# #Transform the product_name & sale_date columns
# sales_df = sales_df.select("sale_id", trim(lower(col("product_name"))).alias("product_name"),date_format(col("sale_date"), "YYYY-MM").alias("sale_date"))

# # Calculate the total number of times the product was sold using a window function
# saleWindow = Window.partitionBy("product_name", "sale_date")
# sales_df_final = sales_df.withColumn("saleCount", count("sale_id").over(saleWindow))

# # Select and show the result columns
# result_df = sales_df_final.select("product_name", "sale_date", "saleCount") \
#     .orderBy("product_name", "sale_date")

# result_df.show()

In [49]:
columns = ["sale_id" , "product_name" , "sale_date"]

data = [(1, "LCPHONE  ","2000-01-16"),
(2,"LCPhone  ","2000-01-17"),
(3,"LcPhOnE  ","2000-02-18"),
(4,"LCKeyCHAiN ","2000-02-19"),
(5,"LCKeyChain","2000-02-28"),
(6,"Matryoshka","2000-03-31")]

df = spark.createDataFrame(data, schema=columns)
df.show()

+-------+------------+----------+
|sale_id|product_name| sale_date|
+-------+------------+----------+
|      1|   LCPHONE  |2000-01-16|
|      2|   LCPhone  |2000-01-17|
|      3|   LcPhOnE  |2000-02-18|
|      4| LCKeyCHAiN |2000-02-19|
|      5|  LCKeyChain|2000-02-28|
|      6|  Matryoshka|2000-03-31|
+-------+------------+----------+



In [50]:
# 1. convert product_name in lowercase without leading or trailing white spaces.

trimdata = df.withColumn("product_name", trim(lower(df.product_name)))
trimdata.show()

# 2. set sale_date in the format ('YYYY-MM').
datadf = trimdata.withColumn("sales_data", date_format(col("sale_date"), "yyyy-MM"))
# trimdata.selectExpr("date_format(sale_date, 'YYYY-MM')")
datadf.show()

# 3. calculate total the number of times the product was sold in this month.
window = Window.partitionBy("product_name", "sales_data")
sales_final = datadf.withColumn("salesCount", count("sale_id").over(window)).orderBy("product_name", "sale_date")
sales_final.show()

result_df = sales_final.select("product_name", "sale_date", "salesCount")
result_df.show()

+-------+------------+----------+
|sale_id|product_name| sale_date|
+-------+------------+----------+
|      1|     lcphone|2000-01-16|
|      2|     lcphone|2000-01-17|
|      3|     lcphone|2000-02-18|
|      4|  lckeychain|2000-02-19|
|      5|  lckeychain|2000-02-28|
|      6|  matryoshka|2000-03-31|
+-------+------------+----------+

+-------+------------+----------+----------+
|sale_id|product_name| sale_date|sales_data|
+-------+------------+----------+----------+
|      1|     lcphone|2000-01-16|   2000-01|
|      2|     lcphone|2000-01-17|   2000-01|
|      3|     lcphone|2000-02-18|   2000-02|
|      4|  lckeychain|2000-02-19|   2000-02|
|      5|  lckeychain|2000-02-28|   2000-02|
|      6|  matryoshka|2000-03-31|   2000-03|
+-------+------------+----------+----------+

+-------+------------+----------+----------+----------+
|sale_id|product_name| sale_date|sales_data|salesCount|
+-------+------------+----------+----------+----------+
|      4|  lckeychain|2000-02-19|   2000

In [51]:
# https://www.linkedin.com/feed/update/urn:li:activity:7110465175123632128/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7110465175123632128%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
# Cummulative is nothing but a "Running Sum" of salary column.

data= [("James", "Sales", 2000),("sofy", "Sales", 3000),("Laren", "Sales", 4000),("Kiku", "Sales", 5000),
       ("Sam", "Finance", 6000),("Samuel", "Finance", 7000),("Yash", "Finance", 8000),
       ("Rabin", "Finance", 9000),("Lukasz", "Marketing", 10000),("Jolly", "Marketing", 11000),
       ("Mausam", "Marketing", 12000),("Lamba", "Marketing", 13000),("Jogesh", "HR", 14000),
       ("Mannu", "HR", 15000),("Sylvia", "HR", 16000),("Sama", "HR", 17000)]

#Schema
# emp_schema = StructType([StructField("name",StringType()), StructField("dept_name",StringType()),StructField("Salary", StringType())])

emp_schema = ["name", "dept_name", "Salary"]
employees_Salary_df = spark.createDataFrame(data,emp_schema)

# employees_Salary_df.show(truncate=False)
employees_Salary_df.show()

+------+---------+------+
|  name|dept_name|Salary|
+------+---------+------+
| James|    Sales|  2000|
|  sofy|    Sales|  3000|
| Laren|    Sales|  4000|
|  Kiku|    Sales|  5000|
|   Sam|  Finance|  6000|
|Samuel|  Finance|  7000|
|  Yash|  Finance|  8000|
| Rabin|  Finance|  9000|
|Lukasz|Marketing| 10000|
| Jolly|Marketing| 11000|
|Mausam|Marketing| 12000|
| Lamba|Marketing| 13000|
|Jogesh|       HR| 14000|
| Mannu|       HR| 15000|
|Sylvia|       HR| 16000|
|  Sama|       HR| 17000|
+------+---------+------+



In [52]:
window_spec = (Window.partitionBy("dept_name").orderBy("salary"))
# window_spec = (Window.partitionBy("dept_name").orderBy("salary").rowsBetween(Window.unboundedPreceding,0))

#Cummulative salary
cum_sum_df = employees_Salary_df.withColumn('cum_sum',sum(col("Salary").cast("Integer")).over(window_spec))

cum_sum_df.show(truncate=False)

+------+---------+------+-------+
|name  |dept_name|Salary|cum_sum|
+------+---------+------+-------+
|Sam   |Finance  |6000  |6000   |
|Samuel|Finance  |7000  |13000  |
|Yash  |Finance  |8000  |21000  |
|Rabin |Finance  |9000  |30000  |
|Jogesh|HR       |14000 |14000  |
|Mannu |HR       |15000 |29000  |
|Sylvia|HR       |16000 |45000  |
|Sama  |HR       |17000 |62000  |
|Lukasz|Marketing|10000 |10000  |
|Jolly |Marketing|11000 |21000  |
|Mausam|Marketing|12000 |33000  |
|Lamba |Marketing|13000 |46000  |
|James |Sales    |2000  |2000   |
|sofy  |Sales    |3000  |5000   |
|Laren |Sales    |4000  |9000   |
|Kiku  |Sales    |5000  |14000  |
+------+---------+------+-------+



In [53]:
# https://www.linkedin.com/feed/update/urn:li:activity:7110563318309810176/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7110563318309810176%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# when to use broadcast function in pyspark?

# # Load the sales_data and customer_info DataFrames
# sales_data = spark.read.csv("dbfs:/FileStore/sales_data.csv", header=True, inferSchema=True)
# customer_info = spark.read.csv("dbfs:/FileStore/customer_info.csv", header=True, inferSchema=True)

# # Optimize the join by broadcasting the smaller DataFrame (customer_info)
# enriched_sales_data = sales_data.join(broadcast(customer_info), "customer_id", "left")

# # Show the resulting DataFrame
# enriched_sales_data.show()


In [54]:
# https://www.linkedin.com/feed/update/urn:li:activity:7110963608590708737/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7110963608590708737%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# Assume you have a DataFrame named 'employee' with columns 'emp_id', 'name', 'dept_id', and 'salary'
# Read your data or replace this with your actual DataFrame creation

# # Add a row number partitioned by department and ordered by salary in descending order

# window_spec = Window.partitionBy("dept_id").orderBy(col("salary").desc())
# employee_with_rank = employee.withColumn("rank", row_number().over(window_spec))

# # employee_with_rank = employee.withColumn("rank", dense_rank().over(window_spec))

# # Filter for 10th highest salary
# tenth_highest_salary = employee_with_rank.filter(col("rank") == 10)

# # Filter for top 6 salaries
# top_six_salaries = employee_with_rank.filter(col("rank").between(1, 6))

# # Show the results
# print("10th Highest Salary in each department:")
# tenth_highest_salary.show()

# print("Top 6 Salaries in each department:")
# top_six_salaries.show()



In [55]:
# https://www.linkedin.com/feed/update/urn:li:activity:7111012314748719104/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7111012314748719104%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29


# 1. **`na.fill()` - PySpark (Spark DataFrame):**

data = [(1, "Alice"), (2, None), (3, "Bob")]
columns = ["ID", "Name"]
df = spark.createDataFrame(data, columns)
df.show()

# Fill null values in the "Name" column with "Unknown"
df.na.fill("Unknown", subset=["Name"]).show()

+---+-----+
| ID| Name|
+---+-----+
|  1|Alice|
|  2| null|
|  3|  Bob|
+---+-----+

+---+-------+
| ID|   Name|
+---+-------+
|  1|  Alice|
|  2|Unknown|
|  3|    Bob|
+---+-------+



In [56]:
# https://www.linkedin.com/feed/update/urn:li:activity:7110958019600912385/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7110958019600912385%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

schema1 = ["name","product_id","units"]
data1 = [("LCHouse1",1,1),
("LCHouse1",2,10),
("LCHouse1",3,5),
("LCHouse2",1,2),
("LCHouse2",2,2),
("LCHouse3",4,1)]

Warehouse = spark.createDataFrame(data1, schema=schema1)
Warehouse.show()

schema2 = ["product_id","product_name","Width","Length","Height"]

data2 = [(1,"LC-TV",5,50,40),
(2,"LC-KeyChain",5,5,5),
(3,"LC-Phone",2,10,10),
(4,"LC-T-Shirt",4,10,20)]

Products = spark.createDataFrame(data2, schema=schema2)
Products.show()

+--------+----------+-----+
|    name|product_id|units|
+--------+----------+-----+
|LCHouse1|         1|    1|
|LCHouse1|         2|   10|
|LCHouse1|         3|    5|
|LCHouse2|         1|    2|
|LCHouse2|         2|    2|
|LCHouse3|         4|    1|
+--------+----------+-----+

+----------+------------+-----+------+------+
|product_id|product_name|Width|Length|Height|
+----------+------------+-----+------+------+
|         1|       LC-TV|    5|    50|    40|
|         2| LC-KeyChain|    5|     5|     5|
|         3|    LC-Phone|    2|    10|    10|
|         4|  LC-T-Shirt|    4|    10|    20|
+----------+------------+-----+------+------+



In [57]:
volumedata = Products.withColumn("volume", col("Width")*col("Length")*col("Height"))
volumedata.show()

+----------+------------+-----+------+------+------+
|product_id|product_name|Width|Length|Height|volume|
+----------+------------+-----+------+------+------+
|         1|       LC-TV|    5|    50|    40| 10000|
|         2| LC-KeyChain|    5|     5|     5|   125|
|         3|    LC-Phone|    2|    10|    10|   200|
|         4|  LC-T-Shirt|    4|    10|    20|   800|
+----------+------------+-----+------+------+------+



In [58]:
joined_data = Warehouse.join(volumedata, Warehouse.product_id == volumedata.product_id, "inner")
joined_data.show()

finaldf = joined_data.groupBy("name").agg(sum(col("volume")).alias("volume")).orderBy("name").select("name", "volume")
finaldf.show()

+--------+----------+-----+----------+------------+-----+------+------+------+
|    name|product_id|units|product_id|product_name|Width|Length|Height|volume|
+--------+----------+-----+----------+------------+-----+------+------+------+
|LCHouse1|         1|    1|         1|       LC-TV|    5|    50|    40| 10000|
|LCHouse2|         1|    2|         1|       LC-TV|    5|    50|    40| 10000|
|LCHouse1|         2|   10|         2| LC-KeyChain|    5|     5|     5|   125|
|LCHouse2|         2|    2|         2| LC-KeyChain|    5|     5|     5|   125|
|LCHouse1|         3|    5|         3|    LC-Phone|    2|    10|    10|   200|
|LCHouse3|         4|    1|         4|  LC-T-Shirt|    4|    10|    20|   800|
+--------+----------+-----+----------+------------+-----+------+------+------+

+--------+------+
|    name|volume|
+--------+------+
|LCHouse1| 10325|
|LCHouse2| 10125|
|LCHouse3|   800|
+--------+------+



In [59]:
# https://www.linkedin.com/feed/update/urn:li:activity:7106478906605535232/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7106478906605535232%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
# How to choose number of executors required for our cluster ?


In [60]:
# https://www.linkedin.com/feed/update/urn:li:activity:7111703324025880576/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7111703324025880576%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

data = [("A", 10),
    ("B", 20),
    ("A", 30),
    ("B", 40),
    ("A", 50)]

columns = ["category", "value"]
df = spark.createDataFrame(data, columns)
df.show()


+--------+-----+
|category|value|
+--------+-----+
|       A|   10|
|       B|   20|
|       A|   30|
|       B|   40|
|       A|   50|
+--------+-----+



In [61]:
# Define a window specification
window_spec = Window.partitionBy("category").orderBy("value")

# Calculate a cumulative sum considering all rows from the start of the partition up to the current row
df.withColumn("cumulative_sum", sum("value").over(window_spec.rowsBetween(Window.unboundedPreceding, Window.currentRow))).show()
df.withColumn("cumulative_sum", sum("value").over(window_spec.rowsBetween(Window.currentRow, Window.unboundedFollowing))).show()
df.withColumn("cumulative_sum", sum("value").over(window_spec.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing))).show()
df.withColumn("cumulative_sum", sum("value").over(window_spec.rowsBetween(Window.currentRow,1))).show()

+--------+-----+--------------+
|category|value|cumulative_sum|
+--------+-----+--------------+
|       A|   10|            10|
|       A|   30|            40|
|       A|   50|            90|
|       B|   20|            20|
|       B|   40|            60|
+--------+-----+--------------+

+--------+-----+--------------+
|category|value|cumulative_sum|
+--------+-----+--------------+
|       A|   10|            90|
|       A|   30|            80|
|       A|   50|            50|
|       B|   20|            60|
|       B|   40|            40|
+--------+-----+--------------+

+--------+-----+--------------+
|category|value|cumulative_sum|
+--------+-----+--------------+
|       A|   10|            90|
|       A|   30|            90|
|       A|   50|            90|
|       B|   20|            60|
|       B|   40|            60|
+--------+-----+--------------+

+--------+-----+--------------+
|category|value|cumulative_sum|
+--------+-----+--------------+
|       A|   10|            40|
|    

In [62]:
# Define a window specification
window_spec = Window.orderBy("value")

# Calculate a cumulative sum considering all rows from the start of the partition up to the current row
df.withColumn("cumulative_sum", sum("value").over(window_spec.rowsBetween(Window.unboundedPreceding, Window.currentRow))).show()
df.withColumn("cumulative_sum", sum("value").over(window_spec.rowsBetween(Window.currentRow, Window.unboundedFollowing))).show()
df.withColumn("cumulative_sum", sum("value").over(window_spec.rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing))).show()
df.withColumn("cumulative_sum", sum("value").over(window_spec.rowsBetween(Window.currentRow,1))).show()

+--------+-----+--------------+
|category|value|cumulative_sum|
+--------+-----+--------------+
|       A|   10|            10|
|       B|   20|            30|
|       A|   30|            60|
|       B|   40|           100|
|       A|   50|           150|
+--------+-----+--------------+

+--------+-----+--------------+
|category|value|cumulative_sum|
+--------+-----+--------------+
|       A|   10|           150|
|       B|   20|           140|
|       A|   30|           120|
|       B|   40|            90|
|       A|   50|            50|
+--------+-----+--------------+

+--------+-----+--------------+
|category|value|cumulative_sum|
+--------+-----+--------------+
|       A|   10|           150|
|       B|   20|           150|
|       A|   30|           150|
|       B|   40|           150|
|       A|   50|           150|
+--------+-----+--------------+

+--------+-----+--------------+
|category|value|cumulative_sum|
+--------+-----+--------------+
|       A|   10|            30|
|    

In [63]:
# inferSchema=True

p = spark.read.format("csv").option("header", "true").option("inferSchema",True)\
    .load(location + "p.csv")
# p.show()
p.printSchema()


# inferSchema=True, by default inferschema is false
p = spark.read.format("csv").option("header", "true").option("inferSchema",False)\
    .load(location + "p.csv")
p.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)



In [64]:
# https://www.linkedin.com/feed/update/urn:li:activity:7111570839325589504/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7111570839325589504%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29


# 1.Given a String and return its transformed form such that it is formed using the count of continuous occurrence of characters.

# Sample Input 1: aaabbcca
# Sample Output 1: a3b2c2a1

# Sample Input 2: aabbbccaa
# Sample Output 2: a2b3c2a2

# Sample Input 3: abcd
# Sample Output 3: a1b1c1d1
    
#     Code:-
def transform_string(input_str):
  result = ""
  i = 0
   
  while i < len(input_str):
    count = 1
    
    # Count continuous occurrences of the current character
    while i + 1 < len(input_str) and input_str[i] == input_str[i + 1]:
      count += 1
      i += 1
        
    # Append the character and its count to the result
    result += input_str[i] + str(count)
    i += 1
   
  return result
print(transform_string("aaabbccaa"))

a3b2c2a2


In [65]:
# https://www.linkedin.com/feed/update/urn:li:activity:7111884516591357954/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7111884516591357954%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
# Explain how and when to use datediff() in pyspark?

from pyspark.sql.types import DateType
data = [
  ('2015-04-08',),
  ('2015-04-09',),
  ('2015-04-10',),
  ('2015-04-11',),
  ('2015-04-12',),
  ('2015-04-13',)
]
columns = ['d1']

schema = StructType([
  StructField("d1", StringType(), True)
])

df = spark.createDataFrame(data, schema=schema)
df.show()
df.printSchema()

# Convert 'd1' to DateType
df = df.withColumn("d1", df["d1"].cast(DateType()))

# Create a new DataFrame with the current date in 'd2'
df_with_current_date = df.withColumn("d2", lit(current_date()))
df_with_current_date.show()

# Calculate the difference in days
df_with_current_date. select(datediff(df_with_current_date.d2, df_with_current_date.d1).alias('diff')).show()


+----------+
|        d1|
+----------+
|2015-04-08|
|2015-04-09|
|2015-04-10|
|2015-04-11|
|2015-04-12|
|2015-04-13|
+----------+

root
 |-- d1: string (nullable = true)

+----------+----------+
|        d1|        d2|
+----------+----------+
|2015-04-08|2023-12-20|
|2015-04-09|2023-12-20|
|2015-04-10|2023-12-20|
|2015-04-11|2023-12-20|
|2015-04-12|2023-12-20|
|2015-04-13|2023-12-20|
+----------+----------+

+----+
|diff|
+----+
|3178|
|3177|
|3176|
|3175|
|3174|
|3173|
+----+



In [66]:
# https://www.linkedin.com/feed/update/urn:li:activity:7110106658030141441/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7110106658030141441%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
data = [  {"user_id": 1, "full_name": "John Doe"},  {"user_id": 2, "full_name": "Jane Smith"},  {"user_id": 3, "full_name": "Michael Johnson"}]

# Create DataFrame
df = spark.createDataFrame(data)
df.show()

+---------------+-------+
|      full_name|user_id|
+---------------+-------+
|       John Doe|      1|
|     Jane Smith|      2|
|Michael Johnson|      3|
+---------------+-------+



In [67]:
splitdata = df.withColumn("first", split("full_name", " ")[0]).withColumn("last", split("full_name", " ")[1])
splitdata.show()

initial_data = splitdata.withColumn("name_initial", expr("concat(substring(first, 1, 1), substring(last, 1, 1))"))
initial_data.show()
initial_data.select("full_name", "user_id", "name_initial").show()

+---------------+-------+-------+-------+
|      full_name|user_id|  first|   last|
+---------------+-------+-------+-------+
|       John Doe|      1|   John|    Doe|
|     Jane Smith|      2|   Jane|  Smith|
|Michael Johnson|      3|Michael|Johnson|
+---------------+-------+-------+-------+

+---------------+-------+-------+-------+------------+
|      full_name|user_id|  first|   last|name_initial|
+---------------+-------+-------+-------+------------+
|       John Doe|      1|   John|    Doe|          JD|
|     Jane Smith|      2|   Jane|  Smith|          JS|
|Michael Johnson|      3|Michael|Johnson|          MJ|
+---------------+-------+-------+-------+------------+

+---------------+-------+------------+
|      full_name|user_id|name_initial|
+---------------+-------+------------+
|       John Doe|      1|          JD|
|     Jane Smith|      2|          JS|
|Michael Johnson|      3|          MJ|
+---------------+-------+------------+



In [68]:
# https://www.linkedin.com/feed/update/urn:li:activity:7110482297321062402/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7110482297321062402%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
#  Extract the first and last names from these emails, which are structured with dots as delimiters. 
data = [
  {"email": "john.doe@email.com"},
  {"email": "jane.smith@email.com"},
  {"email": "alice.wonder@email.com"}
]

# Create DataFrame
df = spark.createDataFrame(data)
df.show()

# # Extract first and last names from email addresses
# df_extracted = df.withColumn("first_name", F.split(F.col("email"), "\\.")[0])
# df_extracted = df_extracted.withColumn("last_name", F.split(F.col("email"), "\\.")[1])
# df_extracted.show(truncate=False)



+--------------------+
|               email|
+--------------------+
|  john.doe@email.com|
|jane.smith@email.com|
|alice.wonder@emai...|
+--------------------+



In [69]:
namedf = df.withColumn("name", split("email", "@")[0])
namedf.show()

namedf.withColumn("first", split("name", "\\.")[0]).withColumn("last", split("name", "\\.")[1]).show()

+--------------------+------------+
|               email|        name|
+--------------------+------------+
|  john.doe@email.com|    john.doe|
|jane.smith@email.com|  jane.smith|
|alice.wonder@emai...|alice.wonder|
+--------------------+------------+

+--------------------+------------+-----+------+
|               email|        name|first|  last|
+--------------------+------------+-----+------+
|  john.doe@email.com|    john.doe| john|   doe|
|jane.smith@email.com|  jane.smith| jane| smith|
|alice.wonder@emai...|alice.wonder|alice|wonder|
+--------------------+------------+-----+------+



In [70]:
# https://www.linkedin.com/feed/update/urn:li:activity:7107967859015770112/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7107967859015770112%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29


In [71]:
# https://www.linkedin.com/feed/update/urn:li:activity:7111946335590744064/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7111946335590744064%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# ❓Write a query to get 𝑻𝒉𝒆 𝒅𝒊𝒇𝒇𝒆𝒓𝒆𝒏𝒄𝒆 𝒐𝒇 𝒂𝒎𝒐𝒖𝒏𝒕 𝒃𝒆𝒕𝒘𝒆𝒆𝒏 𝒂𝒑𝒑𝒍𝒆𝒔 & 𝒐𝒓𝒂𝒏𝒈𝒆𝒔 𝒇𝒐𝒓 𝒆𝒂𝒄𝒉 𝒅𝒂𝒚.


# 📌 DDL Script for Table creation & loading the data
# CREATE TABLE NamasteSQL.Sales_tbl(
# sales_date DATE,
# fruits VARCHAR(25),
# sold_num SMALLINT
# );

# INSERT INTO NamasteSQL.Sales_tbl(sales_date, fruits, sold_num) VALUES
# ('2020-05-01', 'apples', 10),
# ('2020-05-01', 'oranges', 8),
# ('2020-05-02', 'apples', 15),
# ('2020-05-02', 'oranges', 15),
# ('2020-05-03', 'apples', 20),
# ('2020-05-03', 'oranges', 0),
# ('2020-05-04', 'apples', 15),
# ('2020-05-04', 'oranges', 16);

# Select sales_date, diff
# From (Select 
# sales_date, sold_num, Lead(sold_num,1) Over (partition by sales_date order by sales_date) As new_sale_num, (sold_num - Lead(sold_num,1) Over (partition by sales_date order by sales_date)) As diff
# From 
# Sales_tbl) x
# where diff Is not Null;

# select sales_date, 
# (min(case when fruits = 'apples' then sold_num end)- min(case when fruits = 'oranges' then sold_num end)) as diff
# from Sales_tbl
# group by sales_date;

In [72]:
columns = ["sales_date", "fruits", "sold_num"]
data = [('2020-05-01', 'apples', 10),
('2020-05-01', 'oranges', 8),
('2020-05-02', 'apples', 15),
('2020-05-02', 'oranges', 15),
('2020-05-03', 'apples', 20),
('2020-05-03', 'oranges', 0),
('2020-05-04', 'apples', 15),
('2020-05-04', 'oranges', 16)]
df = spark.createDataFrame(data, schema=columns)
df1 = df.orderBy("sales_date", "fruits") # for safer side
df.show()

+----------+-------+--------+
|sales_date| fruits|sold_num|
+----------+-------+--------+
|2020-05-01| apples|      10|
|2020-05-01|oranges|       8|
|2020-05-02| apples|      15|
|2020-05-02|oranges|      15|
|2020-05-03| apples|      20|
|2020-05-03|oranges|       0|
|2020-05-04| apples|      15|
|2020-05-04|oranges|      16|
+----------+-------+--------+



In [73]:
# using Sql
df.createOrReplaceTempView("Sales_tbl")
spark.sql("select sales_date, (min(case when fruits = 'apples' then sold_num end)- min(case when fruits = 'oranges' then sold_num end)) as diff from Sales_tbl group by sales_date;").show()

+----------+----+
|sales_date|diff|
+----------+----+
|2020-05-01|   2|
|2020-05-02|   0|
|2020-05-03|  20|
|2020-05-04|  -1|
+----------+----+



In [74]:
# using DSL
windowSpec = Window.partitionBy("sales_date").orderBy("fruits")

lead_data = df.withColumn("lead", lead("sold_num").over(windowSpec)).filter(col("lead").isNotNull())
lead_data.show()

lead_data.withColumn("diff", col("sold_num")-col("lead")).select("sales_date", "diff").show()

+----------+------+--------+----+
|sales_date|fruits|sold_num|lead|
+----------+------+--------+----+
|2020-05-01|apples|      10|   8|
|2020-05-02|apples|      15|  15|
|2020-05-03|apples|      20|   0|
|2020-05-04|apples|      15|  16|
+----------+------+--------+----+

+----------+----+
|sales_date|diff|
+----------+----+
|2020-05-01|   2|
|2020-05-02|   0|
|2020-05-03|  20|
|2020-05-04|  -1|
+----------+----+



In [75]:
# https://www.linkedin.com/feed/update/urn:li:activity:7112005314522083328/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7112005314522083328%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
# Unlock the real-time potential of your data. Explore Apache Spark's Structured Streaming today!
# ==========================================


# import tempfile
# import time

# with tempfile.TemporaryDirectory() as d, tempfile.TemporaryDirectory() as cp:
#  df = spark.readStream.format("rate").load()
#  q = df.writeStream.format("csv").option("checkpointLocation", cp).start(d)
#  time.sleep(10)
#  q.stop()
#  spark.read.schema("timestamp TIMESTAMP, value STRING").csv(d).show

In [76]:
# https://www.linkedin.com/feed/update/urn:li:activity:7112246904075276289/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7112246904075276289%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

data = [
  (1, 101, 'login', '2023-09-05 08:30:00'),
  (2, 102, 'click', '2023-09-06 12:45:00'),
  (3, 101, 'click', '2023-09-07 14:15:00'),
  (4, 103, 'login', '2023-09-08 09:00:00'),
  (5, 102, 'logout', '2023-09-09 17:30:00'),
  (6, 101, 'click', '2023-09-10 11:20:00'),
  (7, 103, 'click', '2023-09-11 10:15:00'),
  (8, 102, 'click', '2023-09-12 13:10:00')
]

columns = ["log_id", "user_id", "action", "timestamp"]
df = spark.createDataFrame(data, columns)
df.show()

# Calculate the number of actions performed by each user in the last 7 days
result = df.select("user_id", "timestamp") \
  .filter(datediff(current_date(), col("timestamp")) <= 60) \
  .groupBy("user_id") \
  .count() \
  .withColumnRenamed("count", "actions_count")

result.show()


+------+-------+------+-------------------+
|log_id|user_id|action|          timestamp|
+------+-------+------+-------------------+
|     1|    101| login|2023-09-05 08:30:00|
|     2|    102| click|2023-09-06 12:45:00|
|     3|    101| click|2023-09-07 14:15:00|
|     4|    103| login|2023-09-08 09:00:00|
|     5|    102|logout|2023-09-09 17:30:00|
|     6|    101| click|2023-09-10 11:20:00|
|     7|    103| click|2023-09-11 10:15:00|
|     8|    102| click|2023-09-12 13:10:00|
+------+-------+------+-------------------+

+-------+-------------+
|user_id|actions_count|
+-------+-------------+
+-------+-------------+



In [77]:
# https://www.linkedin.com/feed/update/urn:li:activity:7107214877533835264/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7107214877533835264%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
    
data = [("John", "2023-07-01", 5000),("Jane", "2023-07-01", 7000),("John", "2023-08-01", 6000),
        ("Jane", "2023-08-01", 8000),("John", "2023-09-01", 5500),("Jane", "2023-09-01", 7500)]

# Create DataFrame
columns = ["sales_rep", "date", "sales_amount"]
df = spark.createDataFrame(data, columns)
df.show()

# Define the window specification
window_spec = Window.partitionBy("sales_rep").orderBy("date")

# Apply the PySpark Dense_RANK, LAG, and LEAD functions
df_ranked = df.withColumn("quarterly_rank", dense_rank().over(window_spec))
df_analyzed = df_ranked.withColumn("previous_month_sales", lag("sales_amount").over(window_spec))
df_analyzed = df_analyzed.withColumn("next_month_sales", lead("sales_amount").over(window_spec))
df_analyzed.show(truncate=False)

+---------+----------+------------+
|sales_rep|      date|sales_amount|
+---------+----------+------------+
|     John|2023-07-01|        5000|
|     Jane|2023-07-01|        7000|
|     John|2023-08-01|        6000|
|     Jane|2023-08-01|        8000|
|     John|2023-09-01|        5500|
|     Jane|2023-09-01|        7500|
+---------+----------+------------+

+---------+----------+------------+--------------+--------------------+----------------+
|sales_rep|date      |sales_amount|quarterly_rank|previous_month_sales|next_month_sales|
+---------+----------+------------+--------------+--------------------+----------------+
|Jane     |2023-07-01|7000        |1             |null                |8000            |
|Jane     |2023-08-01|8000        |2             |7000                |7500            |
|Jane     |2023-09-01|7500        |3             |8000                |null            |
|John     |2023-07-01|5000        |1             |null                |6000            |
|John     |2023-

In [78]:
# https://www.linkedin.com/feed/update/urn:li:activity:7109377885852868608/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7109377885852868608%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# 🔍 Navigating PySpark: Parsing Nested Strings for Data!⚙️
# 🌱 Example: Extracting Valuable Email Addresses from Complex Text 🚀


# Sample data with nested strings
data = [
  {
    "user_id": 1,
    "bio": "Contact John at john@email.com for inquiries.",
    "posts": [
      "Received an email from jane@email.com today.",
      "Check out our website: www.example.com."
    ]
  },
  {
    "user_id": 2,
    "bio": "Email: support@example.com",
    "posts": [
      "Join our mailing list for updates: news@example.com",
      "Follow us on Twitter: @example"
    ]
  }
]

# Create DataFrame
df = spark.createDataFrame(data)
df.show()


+--------------------+--------------------+-------+
|                 bio|               posts|user_id|
+--------------------+--------------------+-------+
|Contact John at j...|[Received an emai...|      1|
|Email: support@ex...|[Join our mailing...|      2|
+--------------------+--------------------+-------+



In [79]:
# Define a UDF to extract emails using regex
def extract_emails(text):
  return re.findall(r'\S+@\S+', text)

# Extract emails from nested strings
df_extracted = df.withColumn("all_emails", expr("TRANSFORM(posts, x -> split(x, ' '))"))
# df_extracted = df_extracted.withColumn("emails", expr("TRANSFORM(all_emails, x -> TRANSFORM(filter(extract_emails(x), e -> e LIKE '%@%'), y -> regexp_replace(y, '[^a-zA-Z0-9@.]', '')) )"))

df_extracted.show(truncate=False)

+---------------------------------------------+---------------------------------------------------------------------------------------+-------+----------------------------------------------------------------------------------------------------+
|bio                                          |posts                                                                                  |user_id|all_emails                                                                                          |
+---------------------------------------------+---------------------------------------------------------------------------------------+-------+----------------------------------------------------------------------------------------------------+
|Contact John at john@email.com for inquiries.|[Received an email from jane@email.com today., Check out our website: www.example.com.]|1      |[[Received, an, email, from, jane@email.com, today.], [Check, out, our, website:, www.example.com.]]|
|Email: support@exam

In [80]:
# https://www.linkedin.com/posts/iamanuragsharma17_onestepanalytics-pyspark-customerbehavioranalysis-activity-7123526446731386880-5QpO

data = [("C1", "2023-07-01", 150.00),
 ("C1", "2023-07-05", 200.00),
 ("C2", "2023-07-02", 120.00),
 ("C2", "2023-07-03", 80.00)]

# Create DataFrame
columns = ["customer_id", "purchase_date", "purchase_amount"]
df = spark.createDataFrame(data, columns)
df.show()


+-----------+-------------+---------------+
|customer_id|purchase_date|purchase_amount|
+-----------+-------------+---------------+
|         C1|   2023-07-01|          150.0|
|         C1|   2023-07-05|          200.0|
|         C2|   2023-07-02|          120.0|
|         C2|   2023-07-03|           80.0|
+-----------+-------------+---------------+



In [81]:
# Define the window specification
window_spec = Window.partitionBy("customer_id").orderBy("purchase_date")

# Apply Aggregate and First functions
df_with_aggregate = df.withColumn("total_purchases", sum("purchase_amount").over(window_spec))
df_with_aggregate = df_with_aggregate.withColumn("first_purchase_date", first("purchase_date").over(window_spec))
df_with_aggregate = df_with_aggregate.withColumn("recent_purchase_date", last("purchase_date").over(window_spec))

df_with_aggregate.show(truncate=False)

+-----------+-------------+---------------+---------------+-------------------+--------------------+
|customer_id|purchase_date|purchase_amount|total_purchases|first_purchase_date|recent_purchase_date|
+-----------+-------------+---------------+---------------+-------------------+--------------------+
|C1         |2023-07-01   |150.0          |150.0          |2023-07-01         |2023-07-01          |
|C1         |2023-07-05   |200.0          |350.0          |2023-07-01         |2023-07-05          |
|C2         |2023-07-02   |120.0          |120.0          |2023-07-02         |2023-07-02          |
|C2         |2023-07-03   |80.0           |200.0          |2023-07-02         |2023-07-03          |
+-----------+-------------+---------------+---------------+-------------------+--------------------+



In [82]:
# https://www.linkedin.com/feed/update/urn:li:activity:7113486107366854656/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7113486107366854656%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

columns = ["employee_id","team_id"]
data = [(1,8), (2,8), (3,8), (4,7), (5,9), (6,9)]

df = spark.createDataFrame(data, columns)
df.show()

+-----------+-------+
|employee_id|team_id|
+-----------+-------+
|          1|      8|
|          2|      8|
|          3|      8|
|          4|      7|
|          5|      9|
|          6|      9|
+-----------+-------+



In [89]:
result_df = df.groupBy("team_id").agg(count("employee_id").alias("employee_id's"))
result_df.show()

+-------+-------------+
|team_id|employee_id's|
+-------+-------------+
|      8|            3|
|      7|            1|
|      9|            2|
+-------+-------------+



In [90]:
df1 = df.groupBy("team_id").agg(sum('team_id').alias("sum")).withColumnRenamed("team_id", "team_id1")
df1.show()

joined_data = df.join(df1, df.team_id == df1.team_id1, "inner")
joined_data.show()

+--------+---+
|team_id1|sum|
+--------+---+
|       8| 24|
|       7|  7|
|       9| 18|
+--------+---+

+-----------+-------+--------+---+
|employee_id|team_id|team_id1|sum|
+-----------+-------+--------+---+
|          1|      8|       8| 24|
|          2|      8|       8| 24|
|          3|      8|       8| 24|
|          4|      7|       7|  7|
|          5|      9|       9| 18|
|          6|      9|       9| 18|
+-----------+-------+--------+---+



In [91]:
jj = joined_data.withColumn("team_size", (joined_data.sum/joined_data.team_id).cast("integer"))
jj.show()

jj.select("employee_id", "team_size").show()

+-----------+-------+--------+---+---------+
|employee_id|team_id|team_id1|sum|team_size|
+-----------+-------+--------+---+---------+
|          1|      8|       8| 24|        3|
|          2|      8|       8| 24|        3|
|          3|      8|       8| 24|        3|
|          4|      7|       7|  7|        1|
|          5|      9|       9| 18|        2|
|          6|      9|       9| 18|        2|
+-----------+-------+--------+---+---------+

+-----------+---------+
|employee_id|team_size|
+-----------+---------+
|          1|        3|
|          2|        3|
|          3|        3|
|          4|        1|
|          5|        2|
|          6|        2|
+-----------+---------+



In [105]:
schema_ = ["cust_id","first_name","last_name","cust_order","cust_status","full_name","net_salary","age"]
data = [
(12,"Emma","Wilson",8,"Inactive","Emma Wilson",10024,31),
(14,"Isabella","Thomas" ,7,"Inactive" ,"Isabella Thomas",10021,35),
(9,"James","Miller",6,"Active","James Miller",10018,37),
(19,"Daniel","Clark",1,"Active","Daniel Clark",10009,37),
(4,"Emily","Williams" ,2,"Active","Emily Williams",10014,38),
(5,"William" ,"Brown",7,"Inactive","William Brown" ,10001,40),
(8,"Sarah","Davis",9,"Inactive","Sarah Davis",10017,44),
(10,"Olivial","Moore",5,"Inactive","Olivia Moore",10026,44),
(18,"Ava","Lee",6,"Inactive","Ava Lee",10022,47),
(11,"John" ,"Doe",5,"Active","John Doe",10019,48),
(13,"Matthew","Evans" ,3,"Active","Matthew Evans" ,10006,48),
(15,"Joseph" ,"Harris",10,'Active',"Joseph Harris",10017,48),
(21,"Jane","Smith",8,"Inactive","Jane Smith",10009,49)]

In [107]:
# https://www.linkedin.com/feed/update/urn:li:activity:7112367704598085632/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7112367704598085632%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# Define the external source and target paths
source_path = 'your_path' # Update with your actual source file path
target_path = "your_output_path" # Update with your desired target file path

# Extract: Read data from an external CSV file
# df = spark.read.csv(source_path, header=True, schema = 'cust_id int, first_name string,last_name string,cust_order int,cust_status string')


df = spark.createDataFrame(data, schema=schema_)
df.show()

# Transformation 1: Concatenate First and Last Names
df = df.withColumn("full_name", concat(col("first_name"), lit(" "), col("last_name")))

# Transformation 2: Calculate Net Salary (subtract 10% as taxes)
df = df.withColumn("net_salary", floor(lit(10000) + rand() * lit(50)) )
#adding age column
df = df.withColumn("age", floor(lit(20) + rand() * lit(31)))

# Transformation 3: Filter by Age (age >= 30)
df = df.filter(col("age") >= 30)

# Transformation 4: Group by Age and Calculate Average Salary
avg_salary_by_age = df.groupBy("age").agg({"net_salary": "avg"}).withColumnRenamed("avg(salary)", "avg_salary")
# avg_salary_by_age.show()

# Transformation 5: Sort by Age
df = df.orderBy("age")
# df.show()

# # Save the transformed data to an external CSV file
# df.write.csv(target_path, mode="overwrite", header=True)



+-------+----------+---------+----------+-----------+---------------+----------+---+
|cust_id|first_name|last_name|cust_order|cust_status|      full_name|net_salary|age|
+-------+----------+---------+----------+-----------+---------------+----------+---+
|     12|      Emma|   Wilson|         8|   Inactive|    Emma Wilson|     10024| 31|
|     14|  Isabella|   Thomas|         7|   Inactive|Isabella Thomas|     10021| 35|
|      9|     James|   Miller|         6|     Active|   James Miller|     10018| 37|
|     19|    Daniel|    Clark|         1|     Active|   Daniel Clark|     10009| 37|
|      4|     Emily| Williams|         2|     Active| Emily Williams|     10014| 38|
|      5|   William|    Brown|         7|   Inactive|  William Brown|     10001| 40|
|      8|     Sarah|    Davis|         9|   Inactive|    Sarah Davis|     10017| 44|
|     10|   Olivial|    Moore|         5|   Inactive|   Olivia Moore|     10026| 44|
|     18|       Ava|      Lee|         6|   Inactive|        Ava 

In [90]:
# # https://www.linkedin.com/feed/update/urn:li:activity:7112971681165438976/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7112971681165438976%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29


# 🔹LAG Function:
# The LAG function allows you to access the value of a column from the previous row within the current row's result set.
# It is useful for calculating differences between consecutive rows.

# Example:
# SELECT OrderDate, SalesAmount,
#  LAG(SalesAmount) OVER (ORDER BY OrderDate) AS PreviousSales
#  FROM SalesData;

# 🔹LEAD Function:
# The LEAD function is similar to LAG but allows you to access the value of a column from the next row within the current row's result set.

# SELECT
#  OrderDate, SalesAmount,
#  LEAD(SalesAmount) OVER (ORDER BY OrderDate) AS NextSales
#  FROM SalesData;

# 🔹RANK and DENSE_RANK Functions:
# The RANK and DENSE_RANK functions assign a ranking to each row based on the specified ordering within the result set.
# RANK assigns the same rank to rows with equal values, leaving gaps if there are duplicate values.
# DENSE_RANK assigns the same rank to rows with equal values but doesn't leave gaps for duplicates.

# SELECT
#  EmployeeName, Salary,
#  RANK() OVER (ORDER BY Salary DESC) AS SalaryRank,
#  DENSE_RANK() OVER (ORDER BY Salary DESC) AS DenseRank
#  FROM Employees;

# 🔹ROW_NUMBER Function:
# The ROW_NUMBER function assigns a unique sequential number to each row within the result set, without regard to the values in the columns being ordered.

# SELECT
#  ProductName, Price,
#  ROW_NUMBER() OVER (ORDER BY ProductName) AS RowNum
#  FROM Products;

In [91]:
# https://www.linkedin.com/feed/update/urn:li:activity:7113036095340052480/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7113036095340052480%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# Sample stock data
data = [("HDFC", "2023-07-01", 150.00),
    ("HDFC", "2023-07-02", 155.00),
    ("HDFC", "2023-07-03", 160.00)]

# Create DataFrame
columns = ["stock_symbol", "date", "closing_price"]
df = spark.createDataFrame(data, columns)
df.show()

# Define the window specification
window_spec = Window.partitionBy("stock_symbol").orderBy("date")

# Apply the PySpark LAG function
df_with_lag = df.withColumn("previous_day_price", lag("closing_price").over(window_spec))
df_with_lag = df_with_lag.withColumn("price_change", col("closing_price") - col("previous_day_price"))
df_with_lag.show(truncate=False)


+------------+----------+-------------+
|stock_symbol|      date|closing_price|
+------------+----------+-------------+
|        HDFC|2023-07-01|        150.0|
|        HDFC|2023-07-02|        155.0|
|        HDFC|2023-07-03|        160.0|
+------------+----------+-------------+

+------------+----------+-------------+------------------+------------+
|stock_symbol|date      |closing_price|previous_day_price|price_change|
+------------+----------+-------------+------------------+------------+
|HDFC        |2023-07-01|150.0        |null              |null        |
|HDFC        |2023-07-02|155.0        |150.0             |5.0         |
|HDFC        |2023-07-03|160.0        |155.0             |5.0         |
+------------+----------+-------------+------------------+------------+



https://www.linkedin.com/feed/update/urn:li:activity:7110661306436759552/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7110661306436759552%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# Handling NULLS With Pyspark:

In PySpark, the `na` attribute provides various methods to deal with missing or null values in a DataFrame. Here are some common ways to use the `na` attribute:

1. **Dropping Rows with Null Values:**
  - `na.drop()`: Removes rows containing any null values in any column.
  - `na.drop("all")`: Removes rows where all columns have null values.
  - `na.drop(subset=["column_name1", "column_name2"])`: Removes rows with null values in specific columns.

  Example:
  ```python
  df.na.drop()
  df.na.drop("all")
  df.na.drop(subset=["column_name"])
  ```

2. **Filling Null Values:**
  - `na.fill(value, subset=["column_name"])`: Replaces null values in specific columns with the specified value.

  Example:
  ```python
  df.na.fill("replacement_value", subset=["column_name"])
  ```

3. **Replacing Values Conditionally:**
  - `na.replace(map)`: Replaces values in specified columns using a map of replacement values.

  Example:
  ```python
  replace_map = {"column_name1": "new_value1", "column_name2": "new_value2"}
  df.na.replace(replace_map)
  ```

4. **Filling Null Values with Mean/Median:**
  - You can calculate the mean or median of a column and use it to fill null values.

  Example (Filling with Mean):
  ```python
  from pyspark.sql.functions import mean

  mean_value = df.select(mean("column_name")).collect()[0][0]
  df.na.fill(mean_value, subset=["column_name"])
  ```

5. **Dropping Columns with Null Values:**
  - `na.drop("all", subset=["column_name"])`: Drops columns where all values are null.

  Example:
  ```python
  df.na.drop("all", subset=["column_name"])
  ```




In [2]:
# https://www.linkedin.com/feed/update/urn:li:activity:7114176918945476609/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7114176918945476609%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

schema1 = ["employee_id", "department", "salary"]
data1 = [(1,"cse",25000),
(3,"cse",29000),
(5,"ece",22000),
(2,"ece",40000),
(6,"me",12000),
(8,"ece",32000)]

schema2 = ["employee_id", "first_name", "last_name", "DOB", "state", "country"]
data2 = [(1,"dinesh", "H R", "13-04-1995","KA","INDIA"),
(2,"bharath", "kumar", "12-06-1999", "KA", "INDIA"),
(3,"shiva", "kumar", "15-02-1993", "KA", "INDIA"),
(5,"dhivesh", "CTS", "22-05-1999", "KA", "INDIA"),
(6,"santhosh", "CTS", "08-08-1998", "KA", "INDIA"),
(8,"bassappa", "CTS", "07-07-2000","KA", "INDIA")]

employee = spark.createDataFrame(data1, schema=schema1)
employee.show()

employee_details = spark.createDataFrame(data2, schema=schema2)
employee_details.show()


+-----------+----------+------+
|employee_id|department|salary|
+-----------+----------+------+
|          1|       cse| 25000|
|          3|       cse| 29000|
|          5|       ece| 22000|
|          2|       ece| 40000|
|          6|        me| 12000|
|          8|       ece| 32000|
+-----------+----------+------+

+-----------+----------+---------+----------+-----+-------+
|employee_id|first_name|last_name|       DOB|state|country|
+-----------+----------+---------+----------+-----+-------+
|          1|    dinesh|      H R|13-04-1995|   KA|  INDIA|
|          2|   bharath|    kumar|12-06-1999|   KA|  INDIA|
|          3|     shiva|    kumar|15-02-1993|   KA|  INDIA|
|          5|   dhivesh|      CTS|22-05-1999|   KA|  INDIA|
|          6|  santhosh|      CTS|08-08-1998|   KA|  INDIA|
|          8|  bassappa|      CTS|07-07-2000|   KA|  INDIA|
+-----------+----------+---------+----------+-----+-------+



In [3]:
joined_data = employee.join(employee_details, employee.employee_id == employee_details.employee_id, "inner").drop(employee_details.employee_id)
joined_data.show()

+-----------+----------+------+----------+---------+----------+-----+-------+
|employee_id|department|salary|first_name|last_name|       DOB|state|country|
+-----------+----------+------+----------+---------+----------+-----+-------+
|          1|       cse| 25000|    dinesh|      H R|13-04-1995|   KA|  INDIA|
|          2|       ece| 40000|   bharath|    kumar|12-06-1999|   KA|  INDIA|
|          3|       cse| 29000|     shiva|    kumar|15-02-1993|   KA|  INDIA|
|          5|       ece| 22000|   dhivesh|      CTS|22-05-1999|   KA|  INDIA|
|          6|        me| 12000|  santhosh|      CTS|08-08-1998|   KA|  INDIA|
|          8|       ece| 32000|  bassappa|      CTS|07-07-2000|   KA|  INDIA|
+-----------+----------+------+----------+---------+----------+-----+-------+



In [4]:
joined_data = joined_data.withColumn("DOB", to_date(col("DOB"), "dd-MM-yyyy"))
joined_data.show()
joined_data.printSchema()

+-----------+----------+------+----------+---------+----------+-----+-------+
|employee_id|department|salary|first_name|last_name|       DOB|state|country|
+-----------+----------+------+----------+---------+----------+-----+-------+
|          1|       cse| 25000|    dinesh|      H R|1995-04-13|   KA|  INDIA|
|          2|       ece| 40000|   bharath|    kumar|1999-06-12|   KA|  INDIA|
|          3|       cse| 29000|     shiva|    kumar|1993-02-15|   KA|  INDIA|
|          5|       ece| 22000|   dhivesh|      CTS|1999-05-22|   KA|  INDIA|
|          6|        me| 12000|  santhosh|      CTS|1998-08-08|   KA|  INDIA|
|          8|       ece| 32000|  bassappa|      CTS|2000-07-07|   KA|  INDIA|
+-----------+----------+------+----------+---------+----------+-----+-------+

root
 |-- employee_id: long (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- DOB: da

In [5]:
# agedata = joined_data.withColumn('age', datediff(current_date(),col("DOB")))

agedata = joined_data.withColumn('age', floor(datediff(current_date(), to_date(col("DOB")))/365))
agedata.show()

+-----------+----------+------+----------+---------+----------+-----+-------+---+
|employee_id|department|salary|first_name|last_name|       DOB|state|country|age|
+-----------+----------+------+----------+---------+----------+-----+-------+---+
|          1|       cse| 25000|    dinesh|      H R|1995-04-13|   KA|  INDIA| 28|
|          2|       ece| 40000|   bharath|    kumar|1999-06-12|   KA|  INDIA| 24|
|          3|       cse| 29000|     shiva|    kumar|1993-02-15|   KA|  INDIA| 30|
|          5|       ece| 22000|   dhivesh|      CTS|1999-05-22|   KA|  INDIA| 24|
|          6|        me| 12000|  santhosh|      CTS|1998-08-08|   KA|  INDIA| 25|
|          8|       ece| 32000|  bassappa|      CTS|2000-07-07|   KA|  INDIA| 23|
+-----------+----------+------+----------+---------+----------+-----+-------+---+



In [6]:
max_salary = agedata.select(max("salary")).collect()[0][0]
print(max_salary)

40000


In [7]:
salary_diff = agedata.withColumn("salary_diff_to_reach_max", max_salary- col("salary"))
salary_diff.show()

+-----------+----------+------+----------+---------+----------+-----+-------+---+------------------------+
|employee_id|department|salary|first_name|last_name|       DOB|state|country|age|salary_diff_to_reach_max|
+-----------+----------+------+----------+---------+----------+-----+-------+---+------------------------+
|          1|       cse| 25000|    dinesh|      H R|1995-04-13|   KA|  INDIA| 28|                   15000|
|          2|       ece| 40000|   bharath|    kumar|1999-06-12|   KA|  INDIA| 24|                       0|
|          3|       cse| 29000|     shiva|    kumar|1993-02-15|   KA|  INDIA| 30|                   11000|
|          5|       ece| 22000|   dhivesh|      CTS|1999-05-22|   KA|  INDIA| 24|                   18000|
|          6|        me| 12000|  santhosh|      CTS|1998-08-08|   KA|  INDIA| 25|                   28000|
|          8|       ece| 32000|  bassappa|      CTS|2000-07-07|   KA|  INDIA| 23|                    8000|
+-----------+----------+------+------

In [8]:
# first_last = salary_diff.withColumn("full_name", concat_ws(sep = " ", col("first_name"), col("last_name")))
first_last = salary_diff.withColumn("employee_full_name", concat_ws(" ", salary_diff.first_name ,salary_diff.last_name))
first_last.select("employee_id", "employee_full_name").show()


+-----------+------------------+
|employee_id|employee_full_name|
+-----------+------------------+
|          1|        dinesh H R|
|          2|     bharath kumar|
|          3|       shiva kumar|
|          5|       dhivesh CTS|
|          6|      santhosh CTS|
|          8|      bassappa CTS|
+-----------+------------------+



In [9]:
# Everything in one thing.

resultDF = employee\
.join(employee_details, employee.employee_id==employee_details.employee_id, "inner").drop(employee.employee_id)\
.withColumn("employee_full_name", concat_ws(",", col("first_name"), col("last_name")))\
.withColumn("Salary_Diff_to_reach_highest_sal", lit(max_salary) - col("salary"))\
.withColumn("age", floor(datediff(current_date(), to_date(col("DOB")))/365))\
.select("employee_id",
     "employee_full_name",
     "department",
     "salary",
     "Salary_Diff_to_reach_highest_sal",
     "DOB",
     "state",
     "country",
     "age")

resultDF.show()

+-----------+------------------+----------+------+--------------------------------+----------+-----+-------+----+
|employee_id|employee_full_name|department|salary|Salary_Diff_to_reach_highest_sal|       DOB|state|country| age|
+-----------+------------------+----------+------+--------------------------------+----------+-----+-------+----+
|          1|        dinesh,H R|       cse| 25000|                           15000|13-04-1995|   KA|  INDIA|null|
|          2|     bharath,kumar|       ece| 40000|                               0|12-06-1999|   KA|  INDIA|null|
|          3|       shiva,kumar|       cse| 29000|                           11000|15-02-1993|   KA|  INDIA|null|
|          5|       dhivesh,CTS|       ece| 22000|                           18000|22-05-1999|   KA|  INDIA|null|
|          6|      santhosh,CTS|        me| 12000|                           28000|08-08-1998|   KA|  INDIA|null|
|          8|      bassappa,CTS|       ece| 32000|                            8000|07-07

In [10]:
# https://www.linkedin.com/posts/ankur-bhattacharya-10055877_dataengineering-jpmorganinterview-datajobs-activity-7114127851586502656-mXl7?utm_source=share&utm_medium=member_desktop&trk=public_post_comment-text

schema_ = ["PERSON" ,"TYPE" ,"AGE"]

data = [("A1" ,"ADULT" ,54),
("A2" ,"ADULT" ,53),
("A3" ,"ADULT" ,52),
("A4" ,"ADULT" ,58),
("A5" ,"ADULT" ,54),
("C1" ,"CHILD" ,20),
("C2" ,"CHILD" ,19),
("C3" ,"CHILD" ,22),
("C4" ,"CHILD" ,15)]

df = spark.createDataFrame(data, schema=schema_)
df.show()

+------+-----+---+
|PERSON| TYPE|AGE|
+------+-----+---+
|    A1|ADULT| 54|
|    A2|ADULT| 53|
|    A3|ADULT| 52|
|    A4|ADULT| 58|
|    A5|ADULT| 54|
|    C1|CHILD| 20|
|    C2|CHILD| 19|
|    C3|CHILD| 22|
|    C4|CHILD| 15|
+------+-----+---+



In [31]:
adult = df.filter(col("TYPE") == "ADULT").sort(desc(col("AGE")))
adult.show()

child = df.filter(col("TYPE") == "CHILD").sort(asc(col("AGE")))
child.show()

+------+-----+---+
|PERSON| TYPE|AGE|
+------+-----+---+
|    A4|ADULT| 58|
|    A1|ADULT| 54|
|    A5|ADULT| 54|
|    A2|ADULT| 53|
|    A3|ADULT| 52|
+------+-----+---+

+------+-----+---+
|PERSON| TYPE|AGE|
+------+-----+---+
|    C4|CHILD| 15|
|    C2|CHILD| 19|
|    C1|CHILD| 20|
|    C3|CHILD| 22|
+------+-----+---+



In [50]:
window_spec = Window.orderBy(desc("AGE"))
window_spec1 = Window.orderBy("AGE")

adult_row = adult.withColumn("row_number", row_number().over(window_spec))
adult_row.show()

child_row = child.withColumn("row_number", row_number().over(window_spec1))\
            .withColumnRenamed("PERSON","CHILD")
child_row.show()

+------+-----+---+----------+
|PERSON| TYPE|AGE|row_number|
+------+-----+---+----------+
|    A4|ADULT| 58|         1|
|    A1|ADULT| 54|         2|
|    A5|ADULT| 54|         3|
|    A2|ADULT| 53|         4|
|    A3|ADULT| 52|         5|
+------+-----+---+----------+

+-----+-----+---+----------+
|CHILD| TYPE|AGE|row_number|
+-----+-----+---+----------+
|   C4|CHILD| 15|         1|
|   C2|CHILD| 19|         2|
|   C1|CHILD| 20|         3|
|   C3|CHILD| 22|         4|
+-----+-----+---+----------+



In [51]:
finaldf = adult_row.join(child_row, adult_row.row_number == child_row.row_number, "full")
finaldf.show()
                        
finaldf.selectExpr("PERSON as ADULT", "CHILD").show()

+------+-----+---+----------+-----+-----+----+----------+
|PERSON| TYPE|AGE|row_number|CHILD| TYPE| AGE|row_number|
+------+-----+---+----------+-----+-----+----+----------+
|    A4|ADULT| 58|         1|   C4|CHILD|  15|         1|
|    A1|ADULT| 54|         2|   C2|CHILD|  19|         2|
|    A5|ADULT| 54|         3|   C1|CHILD|  20|         3|
|    A2|ADULT| 53|         4|   C3|CHILD|  22|         4|
|    A3|ADULT| 52|         5| null| null|null|      null|
+------+-----+---+----------+-----+-----+----+----------+

+-----+-----+
|ADULT|CHILD|
+-----+-----+
|   A4|   C4|
|   A1|   C2|
|   A5|   C1|
|   A2|   C3|
|   A3| null|
+-----+-----+



In [53]:
# PySpark Solution: 
df = spark.createDataFrame(data, schema=schema_)
df.show()

wind_spec=Window.partitionBy(col("type")).orderBy(col("Age").desc())

adult=df.withColumn("rank",row_number().over(wind_spec)).filter(col("type")=="ADULT")
# adult.show()

child=df.withColumn("rank", row_number().over(Window.partitionBy(col("type")).
orderBy(col("Age").asc()))).filter(col("type")=="CHILD")
# child.show()


child=child.select(col("person").alias("child_person"),"type","age","rank")
# child.show()

result=adult.join(child,"rank","left")
# result.show()

result.select(col("person").alias("ADULT"),col("child_person").alias("CHILD")).show()

+------+-----+---+
|PERSON| TYPE|AGE|
+------+-----+---+
|    A1|ADULT| 54|
|    A2|ADULT| 53|
|    A3|ADULT| 52|
|    A4|ADULT| 58|
|    A5|ADULT| 54|
|    C1|CHILD| 20|
|    C2|CHILD| 19|
|    C3|CHILD| 22|
|    C4|CHILD| 15|
+------+-----+---+

+------+-----+---+----+
|PERSON| TYPE|AGE|rank|
+------+-----+---+----+
|    A4|ADULT| 58|   1|
|    A1|ADULT| 54|   2|
|    A5|ADULT| 54|   3|
|    A2|ADULT| 53|   4|
|    A3|ADULT| 52|   5|
+------+-----+---+----+

+------+-----+---+----+
|PERSON| TYPE|AGE|rank|
+------+-----+---+----+
|    C4|CHILD| 15|   1|
|    C2|CHILD| 19|   2|
|    C1|CHILD| 20|   3|
|    C3|CHILD| 22|   4|
+------+-----+---+----+

+------------+-----+---+----+
|child_person| type|age|rank|
+------------+-----+---+----+
|          C4|CHILD| 15|   1|
|          C2|CHILD| 19|   2|
|          C1|CHILD| 20|   3|
|          C3|CHILD| 22|   4|
+------------+-----+---+----+

+----+------+-----+---+------------+-----+----+
|rank|PERSON| TYPE|AGE|child_person| type| age|
+----

In [5]:
# https://www.linkedin.com/feed/update/urn:li:activity:7114127851586502656/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7114127851586502656%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

schema_ = ["PERSON" ,"TYPE" ,"AGE"]

data = [("A1" ,"ADULT" ,54),
("A2" ,"ADULT" ,53),
("A3" ,"ADULT" ,52),
("A4" ,"ADULT" ,58),
("A5" ,"ADULT" ,54),
("C1" ,"CHILD" ,20),
("C2" ,"CHILD" ,19),
("C3" ,"CHILD" ,22),
("C4" ,"CHILD" ,15)]

df = spark.createDataFrame(data, schema=schema_)
df.show()

+------+-----+---+
|PERSON| TYPE|AGE|
+------+-----+---+
|    A1|ADULT| 54|
|    A2|ADULT| 53|
|    A3|ADULT| 52|
|    A4|ADULT| 58|
|    A5|ADULT| 54|
|    C1|CHILD| 20|
|    C2|CHILD| 19|
|    C3|CHILD| 22|
|    C4|CHILD| 15|
+------+-----+---+



In [6]:
df.createOrReplaceTempView("persons")
spark.sql("select * from persons").show()

+------+-----+---+
|PERSON| TYPE|AGE|
+------+-----+---+
|    A1|ADULT| 54|
|    A2|ADULT| 53|
|    A3|ADULT| 52|
|    A4|ADULT| 58|
|    A5|ADULT| 54|
|    C1|CHILD| 20|
|    C2|CHILD| 19|
|    C3|CHILD| 22|
|    C4|CHILD| 15|
+------+-----+---+



In [10]:
# spark.sql("with cte1 as (\
# select *, case \
# when type='adult' then dense_rank() over (partition by type order by age desc)\
# when type='child' then dense_rank() over (partition by type order by age)\
# end as rn from person)\
# select p1.person as adult, p2.person as child from cte1 p1 join cte1 p2 on p1.rn=p2.rn \
# and p1.type<>p2.type and p1.type='adult'").show()

In [11]:
# https://www.linkedin.com/feed/update/urn:li:activity:7114120072947757056/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7114120072947757056%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# The salary is incremented by following criteria:
# Managers -> 20%
# Leads -> 15%
# Senior -> 10%
# Junior -> 20%

schema_ = ["first_name","last_name","designation","salary"]

data = [("Pat","Given","Junior",32720.55),
("Remy","Esch","Senior",569929.25),
("Rafaello","Worrall","Manager",622487.93),
("Gherardo","Chadbourne","Lead",425927.70)]

df = spark.createDataFrame(data, schema=schema_)
df.show()


+----------+----------+-----------+---------+
|first_name| last_name|designation|   salary|
+----------+----------+-----------+---------+
|       Pat|     Given|     Junior| 32720.55|
|      Remy|      Esch|     Senior|569929.25|
|  Rafaello|   Worrall|    Manager|622487.93|
|  Gherardo|Chadbourne|       Lead| 425927.7|
+----------+----------+-----------+---------+



In [18]:
salary_incr = df.withColumn("salary_inc", expr("case when designation == 'Manager' then round(salary + salary * (20/100), 2) \
              when designation == 'Lead' then round(salary + salary * (15/100), 2)\
              when designation == 'Senior' then round(salary + salary * (10/100), 2)\
              when designation == 'Junior' then round(salary + salary * (20/100), 2)\
                else 0 end"))
salary_incr.show()

+----------+----------+-----------+---------+----------+
|first_name| last_name|designation|   salary|salary_inc|
+----------+----------+-----------+---------+----------+
|       Pat|     Given|     Junior| 32720.55|  39264.66|
|      Remy|      Esch|     Senior|569929.25| 626922.18|
|  Rafaello|   Worrall|    Manager|622487.93| 746985.52|
|  Gherardo|Chadbourne|       Lead| 425927.7| 489816.86|
+----------+----------+-----------+---------+----------+



In [27]:
# https://www.linkedin.com/posts/hasnain-motagamwala_players-tournament-win-results-code-activity-7123234638797373440-1Obc

# Matches Data
matches_list = [(2017, 2, 1, 1, 2),(2018, 3, 1, 3, 2),(2019, 3, 1, 1, 3)]
matches_header = ["year", "Wimbledon", "Fr_Open", "US_Open", "Au_Open"]

# Player Data
players_list = [(1, "Nadal"),(2, "Federer"),(3, "Novak")]
players_header = ["Player_ID", "player_Name"]

matches = spark.createDataFrame(matches_list, matches_header)
matches.show()

players = spark.createDataFrame(players_list, players_header)
players.show()

+----+---------+-------+-------+-------+
|year|Wimbledon|Fr_Open|US_Open|Au_Open|
+----+---------+-------+-------+-------+
|2017|        2|      1|      1|      2|
|2018|        3|      1|      3|      2|
|2019|        3|      1|      1|      3|
+----+---------+-------+-------+-------+

+---------+-----------+
|Player_ID|player_Name|
+---------+-----------+
|        1|      Nadal|
|        2|    Federer|
|        3|      Novak|
+---------+-----------+



In [29]:
#Used GroupedCount
inter_res= matches.groupBy(col("Wimbledon").alias("PlayerID")).count().union(
matches.groupBy(col("Fr_Open").alias("PlayerID")).count()).union(
matches.groupBy(col("US_Open").alias("PlayerID")).count()).union(
matches.groupBy(col("Au_Open").alias("PlayerID")).count())
inter_res.show()

#Getting the No_of_GrandSlams per player
res = inter_res.groupBy("PlayerID").agg(sum("count").alias("No_of_GrandSlams"))
res.show()

#Player name with player ID
res.join(players, res.PlayerID==players.Player_ID, "inner")\
    .select(col("Player_ID"), col("player_Name"), col("No_of_GrandSlams")).show()

+--------+-----+
|PlayerID|count|
+--------+-----+
|       2|    1|
|       3|    2|
|       1|    3|
|       1|    2|
|       3|    1|
|       2|    2|
|       3|    1|
+--------+-----+

+--------+----------------+
|PlayerID|No_of_GrandSlams|
+--------+----------------+
|       3|               4|
|       2|               3|
|       1|               5|
+--------+----------------+

+---------+-----------+----------------+
|Player_ID|player_Name|No_of_GrandSlams|
+---------+-----------+----------------+
|        1|      Nadal|               5|
|        2|    Federer|               3|
|        3|      Novak|               4|
+---------+-----------+----------------+



In [30]:
# https://www.linkedin.com/feed/update/urn:li:activity:7114435642473820160/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7114435642473820160%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# Sample student data
data = [("Amar", 85),("Akbar", 92),("Anthony", 78),("Lakshmi", 78),("Salma", 97),("Jenny", 70)]

# Create DataFrame
columns = ["student_name", "exam_score"]
df = spark.createDataFrame(data, columns)

# Define the window specification
window_spec = Window.orderBy(col("exam_score").desc())

# Apply the PySpark RANK function
df_with_rank = df.withColumn("rank", rank().over(window_spec))
df_with_rank.show()



+------------+----------+----+
|student_name|exam_score|rank|
+------------+----------+----+
|       Salma|        97|   1|
|       Akbar|        92|   2|
|        Amar|        85|   3|
|     Anthony|        78|   4|
|     Lakshmi|        78|   4|
|       Jenny|        70|   6|
+------------+----------+----+



In [19]:
# https://www.linkedin.com/feed/update/urn:li:activity:7114489959322939392/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7114489959322939392%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# Slowly Changing Dimension Type 2 (SCD Type 2) Using PySpark:

# SCD Type 2 is a common technique used in data warehousing to track changes in dimension data over time. 
# It involves maintaining historical versions of records when a dimension attribute's value changes. 
# In PySpark, you can implement SCD Type 2 by managing historical records and handling updates. 
# Here's a step-by-step demonstration of how to implement SCD Type 2 using PySpark:

# Assuming you have two DataFrames: `currentData` representing the current dimension data and `newData` 
# representing the incoming data that might have changes.


# Sample data for demonstration
currentData = spark.createDataFrame([
  (1, "John", "2021-01-01", "999-123-4567"),
  (2, "Alice", "2021-02-01", "888-987-6543")
], ["id", "name", "valid_from", "phone"])

newData = spark.createDataFrame([
  (1, "John Doe", "2021-06-15", "999-123-4567"),
  (3, "Bob", "2021-05-10", "777-555-1234")
], ["id", "name", "valid_from", "phone"])

# Add necessary columns to the DataFrames
currentData = currentData.withColumn("valid_to", current_date())
currentData.show()

newData = newData.withColumn("valid_from", newData["valid_from"].cast("date"))
newData = newData.withColumn("valid_to", lit(None).cast("date"))
newData.show()

# Define a union of the currentData and newData
unionedData = currentData.union(newData)
unionedData.show()

# Implement SCD Type 2 logic

# Create a window specification for partitioning and ordering
windowSpec = Window.partitionBy("id").orderBy("valid_from")

# Add a new column to track the change sequence
changeSequence = unionedData.withColumn("change_sequence", row_number().over(windowSpec))
changeSequence.show()

# Filter out records where change_sequence is greater than 1 (i.e., duplicates)
scd2Data = changeSequence.filter(changeSequence.change_sequence == 1)

# Show the SCD Type 2 result
scd2Data.show()


ConnectionRefusedError: [Errno 61] Connection refused

In [5]:
# https://www.linkedin.com/feed/update/urn:li:activity:7114576934088376320/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7114576934088376320%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
# 𝗤𝘂𝗲𝘀𝘁𝗶𝗼𝗻: Find the date on which maximum number of rooms were searched along with their count.

schema_ = ["user_id", "date_searched", "filter_room_types"]

data = [(1 , "2022-01-01" , "entire home,private room"),
(2 , "2022-01-02" , "entire home,shared room"),
(3 , "2022-01-02" , "private room,shared room"),
(4 , "2022-01-03" , "private room")]

df = spark.createDataFrame(data, schema=schema_)
df.show(truncate=False)

+-------+-------------+------------------------+
|user_id|date_searched|filter_room_types       |
+-------+-------------+------------------------+
|1      |2022-01-01   |entire home,private room|
|2      |2022-01-02   |entire home,shared room |
|3      |2022-01-02   |private room,shared room|
|4      |2022-01-03   |private room            |
+-------+-------------+------------------------+



In [6]:
split_data = df.withColumn("filter_room_types", explode(split(df["filter_room_types"], ",")))
split_data.show()

+-------+-------------+-----------------+
|user_id|date_searched|filter_room_types|
+-------+-------------+-----------------+
|      1|   2022-01-01|      entire home|
|      1|   2022-01-01|     private room|
|      2|   2022-01-02|      entire home|
|      2|   2022-01-02|      shared room|
|      3|   2022-01-02|     private room|
|      3|   2022-01-02|      shared room|
|      4|   2022-01-03|     private room|
+-------+-------------+-----------------+



In [13]:
groupedData = split_data.groupBy("date_searched").agg(
    collect_list("filter_room_types").alias("All Rooms"), 
    count("filter_room_types").alias("count")).orderBy(col("count").desc())
groupedData.show()

resultDf.limit(1).show(truncate=False)

+-------------+--------------------+-----+
|date_searched|           All Rooms|count|
+-------------+--------------------+-----+
|   2022-01-02|[entire home, sha...|    4|
|   2022-01-01|[entire home, pri...|    2|
|   2022-01-03|      [private room]|    1|
+-------------+--------------------+-----+

+-------------+-----------------------------------------------------+-----+
|date_searched|All Rooms                                            |count|
+-------------+-----------------------------------------------------+-----+
|2022-01-02   |[entire home, shared room, private room, shared room]|4    |
+-------------+-----------------------------------------------------+-----+



https://www.linkedin.com/feed/update/urn:li:activity:7114549577948528640/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7114549577948528640%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

## Topic : 🚀 Harnessing the Power of Lazy Evaluation in Apache Spark! 🌟


Topic : 🚀 Harnessing the Power of Lazy Evaluation in Apache Spark! 🌟

Lazy evaluation in Apache Spark is like a smart assistant for data processing. Imagine you have a magical helper who doesn't do any work until it's really needed, making things faster and giving you lots of options.

For example, when working with lots of data, instead of doing everything at once, this helper only does each task when you say, "I need this now!" This way, you save time and resources because you're not doing unnecessary work.

### Sample data
data = [("Alice", 25), ("Bob", 30), ("Charlie", 22), ("David", 28), ("Eve", 35)]

df = spark.createDataFrame(data, ["Name", "Age"])

### Transformation 1: Group by Age
grouped_data = df.groupBy("Age").count()

### Transformation 2: Filter Ages above 25
filtered_data = grouped_data.filter(grouped_data.Age > 25)

### Action 1: Show the grouped data (Lazy Evaluation)
print("Grouped Data (Lazy):")

grouped_data.show()

### Action 2: Show the filtered data (Lazy Evaluation)
print("Filtered Data (Lazy):")

filtered_data.show()

In this code:

1.We create a Spark session.

2.We create a DataFrame df with sample data containing names and ages.

3.Transformation 1 groups the data by age and counts the occurrences but does not execute it immediately.

4.Transformation 2 filters the grouped data to select ages above 25 but also does not execute it immediately.

5.Action 1 displays the grouped data using .show(), triggering the execution of the grouping transformation.

6.Action 2 displays the filtered data using .show(), triggering the execution of both the grouping and filtering transformations.

The use of groupBy and filtering with filter demonstrates how different transformations can be applied lazily, and execution is deferred until an action is triggered.

✨ Why Lazy Evaluation Matters ✨

🎯 Optimization: Spark fine-tunes the execution plan based on the entire sequence of transformations, avoiding unnecessary work and improving efficiency.

🚀 Efficiency: It minimizes data shuffling, cuts down on disk reads, and supercharges overall performance.

🤝 Flexibility: You can craft intricate data pipelines without prematurely incurring computational costs, a true game-changer in the world of big data.

https://www.linkedin.com/feed/update/urn:li:activity:7115146007184609280/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115146007184609280%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29


PySpark Interview Questions 🐉

Question 16 : What is difference between narrow and wide transforamtion in PySpark ?

In PySpark, transformations are categorized into two types: narrow transformations and wide transformations. These categories are based on how they impact the execution plan and data shuffling in a Spark job.

Narrow Transformations:
Narrow transformations are those transformations where each output partition depends on a single input partition.
They do not require data shuffling or data movement across partitions, making them more efficient.
Examples of narrow transformations include map, filter, and union.

### Sample DataFrame
data = [("Alice", 25), ("Bob", 30), ("Charlie", 22), ("David", 28), ("Eve", 35)]

df = spark.createDataFrame(data, ["Name", "Age"])

### Narrow Transformation: Filtering ages above 25

filtered_df = df.filter(df.Age > 25)

Wide Transformations:
Wide transformations are those transformations where each output partition depends on multiple input partitions.
They require data shuffling or redistribution across partitions, which can be resource-intensive and time-consuming.
Examples of wide transformations include groupByKey and join.

### Sample DataFrames
df1 = spark.createDataFrame([(1, "Alice"), (2, "Bob"), (3, "Charlie")], ["ID", "Name"])

df2 = spark.createDataFrame([(1, "Math"), (2, "Science"), (3, "History")], ["ID", "Subject"])

### Wide Transformation: Joining two DataFrames
joined_df = df1.join(df2, "ID")

Understanding these transformation types helps us optimize our PySpark jobs for blazing-fast data processing! 🔥


In [21]:
# https://www.linkedin.com/feed/update/urn:li:activity:7115214890214002688/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115214890214002688%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

schema_ = ["emp_name","dep_id","salary"]

data = [("Genece", 2, 75000),
 ("Jaimin", 2, 80000),
 ("Pankaj", 2, 80000),
 ("Tarvares", 2, 70000),
 ("Marlania", 4, 70000),
 ("Briana", 4, 85000),
 ("Kimberli", 4, 55000),
 ("Gabriella", 4, 55000),
 ("Lakken", 5, 60000),
 ("Latoynia", 5, 65000)]

df = spark.createDataFrame(data, schema=schema_)
df.show()

+---------+------+------+
| emp_name|dep_id|salary|
+---------+------+------+
|   Genece|     2| 75000|
|   Jaimin|     2| 80000|
|   Pankaj|     2| 80000|
| Tarvares|     2| 70000|
| Marlania|     4| 70000|
|   Briana|     4| 85000|
| Kimberli|     4| 55000|
|Gabriella|     4| 55000|
|   Lakken|     5| 60000|
| Latoynia|     5| 65000|
+---------+------+------+



In [22]:
window_spec = Window.partitionBy("dep_id").orderBy(desc("salary"))

df1 = df.withColumn("salaryRank", rank().over(window_spec))
df1.show()

+---------+------+------+----------+
| emp_name|dep_id|salary|salaryRank|
+---------+------+------+----------+
|   Jaimin|     2| 80000|         1|
|   Pankaj|     2| 80000|         1|
|   Genece|     2| 75000|         3|
| Tarvares|     2| 70000|         4|
|   Briana|     4| 85000|         1|
| Marlania|     4| 70000|         2|
| Kimberli|     4| 55000|         3|
|Gabriella|     4| 55000|         3|
| Latoynia|     5| 65000|         1|
|   Lakken|     5| 60000|         2|
+---------+------+------+----------+



In [23]:
df2 = df1.withColumn("lastRank", max("salaryRank").over(Window.partitionBy("dep_id")))
df2.show()

+---------+------+------+----------+--------+
| emp_name|dep_id|salary|salaryRank|lastRank|
+---------+------+------+----------+--------+
|   Jaimin|     2| 80000|         1|       4|
|   Pankaj|     2| 80000|         1|       4|
|   Genece|     2| 75000|         3|       4|
| Tarvares|     2| 70000|         4|       4|
|   Briana|     4| 85000|         1|       3|
| Marlania|     4| 70000|         2|       3|
| Kimberli|     4| 55000|         3|       3|
|Gabriella|     4| 55000|         3|       3|
| Latoynia|     5| 65000|         1|       2|
|   Lakken|     5| 60000|         2|       2|
+---------+------+------+----------+--------+



In [25]:
FilterData = df2.filter((df2.salaryRank == 1) | (df2.salaryRank == df2.lastRank))
FilterData.show()

+---------+------+------+----------+--------+
| emp_name|dep_id|salary|salaryRank|lastRank|
+---------+------+------+----------+--------+
|   Jaimin|     2| 80000|         1|       4|
|   Pankaj|     2| 80000|         1|       4|
| Tarvares|     2| 70000|         4|       4|
|   Briana|     4| 85000|         1|       3|
| Kimberli|     4| 55000|         3|       3|
|Gabriella|     4| 55000|         3|       3|
| Latoynia|     5| 65000|         1|       2|
|   Lakken|     5| 60000|         2|       2|
+---------+------+------+----------+--------+



In [26]:
final = FilterData.groupBy("dep_id", "salary").agg(collect_set("emp_name").alias("emp_names"))
final.show(truncate=False)

+------+------+---------------------+
|dep_id|salary|emp_names            |
+------+------+---------------------+
|2     |80000 |[Pankaj, Jaimin]     |
|2     |70000 |[Tarvares]           |
|4     |85000 |[Briana]             |
|4     |55000 |[Gabriella, Kimberli]|
|5     |65000 |[Latoynia]           |
|5     |60000 |[Lakken]             |
+------+------+---------------------+



In [2]:
# https://www.linkedin.com/posts/hasnain-motagamwala_key-takeaways-approach-activity-7114867617609203713-UAMz?utm_source=share&utm_medium=member_desktop

# Sample input data
data = [("Marlania", 1, 92643),
       ("Briana", 1, 87202),
       ("Kimberli", 2, 51407),
       ("Lakken", 2, 88933),
       ("Micaila", 2, 82145),
       ("Tarvares", 3, 82979),
       ("Latoynia", 3, 55729),
       ("Gabriella", 4, 74132),
       ("Medusa", 4, 72551),
       ("Genece", 5, 98877),
       ("Thurley", 5, 65628)]

# Create a DataFrame
df = spark.createDataFrame(data, ["emp_name", "dep_id", "salary"])
df.show()

+---------+------+------+
| emp_name|dep_id|salary|
+---------+------+------+
| Marlania|     1| 92643|
|   Briana|     1| 87202|
| Kimberli|     2| 51407|
|   Lakken|     2| 88933|
|  Micaila|     2| 82145|
| Tarvares|     3| 82979|
| Latoynia|     3| 55729|
|Gabriella|     4| 74132|
|   Medusa|     4| 72551|
|   Genece|     5| 98877|
|  Thurley|     5| 65628|
+---------+------+------+



In [3]:
max_min = df.groupBy("dep_id").agg(max("salary").alias("maxSalary"), min("salary").alias("minSalary"))
max_min.show()

joinedData = df.withColumnRenamed("dep_id", "dept_id")
joinedData = joinedData.join(max_min, joinedData.dept_id == max_min.dep_id, "inner") 
joinedData.show()

+------+---------+---------+
|dep_id|maxSalary|minSalary|
+------+---------+---------+
|     1|    92643|    87202|
|     2|    88933|    51407|
|     3|    82979|    55729|
|     4|    74132|    72551|
|     5|    98877|    65628|
+------+---------+---------+

+---------+-------+------+------+---------+---------+
| emp_name|dept_id|salary|dep_id|maxSalary|minSalary|
+---------+-------+------+------+---------+---------+
| Marlania|      1| 92643|     1|    92643|    87202|
|   Briana|      1| 87202|     1|    92643|    87202|
| Kimberli|      2| 51407|     2|    88933|    51407|
|   Lakken|      2| 88933|     2|    88933|    51407|
|  Micaila|      2| 82145|     2|    88933|    51407|
| Tarvares|      3| 82979|     3|    82979|    55729|
| Latoynia|      3| 55729|     3|    82979|    55729|
|Gabriella|      4| 74132|     4|    74132|    72551|
|   Medusa|      4| 72551|     4|    74132|    72551|
|   Genece|      5| 98877|     5|    98877|    65628|
|  Thurley|      5| 65628|     5|   

In [4]:
resultDF = joinedData.groupBy("dep_id").agg(
    max(when(col("salary") == col("maxSalary"), col("emp_name")))\
    .alias("max_emp"),
    min(when(col("salary") == col("minSalary"), col("emp_name"))\
        .otherwise("null")).alias("min_emp"))
resultDF.show()

+------+---------+--------+
|dep_id|  max_emp| min_emp|
+------+---------+--------+
|     1| Marlania|  Briana|
|     2|   Lakken|Kimberli|
|     3| Tarvares|Latoynia|
|     4|Gabriella|  Medusa|
|     5|   Genece| Thurley|
+------+---------+--------+



In [10]:
# Second Approach

fullData = joinedData.withColumn("max_emp", expr("case when salary == maxSalary then emp_name else null end"))\
.withColumn("min_emp", 
        expr("case when salary == minSalary then emp_name else null end"))
# fullData.show()

res = fullData.select("dept_id", "max_emp", "min_emp")
res.show()

+-------+---------+--------+
|dept_id|  max_emp| min_emp|
+-------+---------+--------+
|      1| Marlania|    null|
|      1|     null|  Briana|
|      2|     null|Kimberli|
|      2|   Lakken|    null|
|      2|     null|    null|
|      3| Tarvares|    null|
|      3|     null|Latoynia|
|      4|Gabriella|    null|
|      4|     null|  Medusa|
|      5|   Genece|    null|
|      5|     null| Thurley|
+-------+---------+--------+



In [14]:
res1 = res.filter(res.max_emp != "null").drop("min_emp")
res1.show()

res2 = res.filter(res.min_emp != "null").withColumnRenamed("dept_id", "dep_id")\
        .drop("max_emp")
res2.show()

final = res1.join(res2, res1.dept_id == res2.dep_id, "inner").drop("dep_id")
final.show()

+-------+---------+
|dept_id|  max_emp|
+-------+---------+
|      1| Marlania|
|      2|   Lakken|
|      3| Tarvares|
|      4|Gabriella|
|      5|   Genece|
+-------+---------+

+------+--------+
|dep_id| min_emp|
+------+--------+
|     1|  Briana|
|     2|Kimberli|
|     3|Latoynia|
|     4|  Medusa|
|     5| Thurley|
+------+--------+

+-------+---------+--------+
|dept_id|  max_emp| min_emp|
+-------+---------+--------+
|      1| Marlania|  Briana|
|      2|   Lakken|Kimberli|
|      3| Tarvares|Latoynia|
|      4|Gabriella|  Medusa|
|      5|   Genece| Thurley|
+-------+---------+--------+



In [18]:
# https://www.linkedin.com/feed/update/urn:li:activity:7115269785868795904/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115269785868795904%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# Challenge : 
# Write a PySpark code for getting difference of average rating and second highest rating for the given dataset. 

Schema = StructType([
    StructField("EmpName", StringType(), True),
    StructField("EmpID", StringType(), True),
    StructField("Department", StringType(), True),
    StructField("Year", IntegerType(), True),
    StructField("Rating", DoubleType(), True)
])

Data = [
    ("Peter", "E5489", "Marketing", 2020, 8.0),
    ("Peter", "E5489", "Marketing", 2019, 7.0),
    ("Peter", "E5489", "Marketing", 2018, 6.5),
    ("Peter", "E5489", "Marketing", 2017, 8.5),
    ("Peter", "E5489", "Marketing", 2016, 7.0),
    ("Philip", "E1027", "HR", 2020, 8.0),
    ("Philip", "E1027", "HR", 2019, 8.5),
    ("Shirley", "E0589", "Finance", 2020, 7.0),
    ("Shirley", "E0589", "Finance", 2019, 5.0),
    ("Shirley", "E0589", "Finance", 2018, 6.0)
]

df = spark.createDataFrame(Data, Schema)
df.show()

+-------+-----+----------+----+------+
|EmpName|EmpID|Department|Year|Rating|
+-------+-----+----------+----+------+
|  Peter|E5489| Marketing|2020|   8.0|
|  Peter|E5489| Marketing|2019|   7.0|
|  Peter|E5489| Marketing|2018|   6.5|
|  Peter|E5489| Marketing|2017|   8.5|
|  Peter|E5489| Marketing|2016|   7.0|
| Philip|E1027|        HR|2020|   8.0|
| Philip|E1027|        HR|2019|   8.5|
|Shirley|E0589|   Finance|2020|   7.0|
|Shirley|E0589|   Finance|2019|   5.0|
|Shirley|E0589|   Finance|2018|   6.0|
+-------+-----+----------+----+------+



In [19]:
# Calculate the average rating for each user
avg_rating = df.groupBy("empid").agg(avg("rating").alias("avgRating"))
avg_rating.show()

+-----+---------+
|empid|avgRating|
+-----+---------+
|E5489|      7.4|
|E1027|     8.25|
|E0589|      6.0|
+-----+---------+



In [20]:
# Applying Dense_rank function over Employee_ID

window_spec = Window.partitionBy("empid").orderBy(desc("rating"))

# Calculate the dense rank of the ratings for each user

dense_rank = df.withColumn("dense_rank", dense_rank().over(window_spec))
dense_rank.show()

+-------+-----+----------+----+------+----------+
|EmpName|EmpID|Department|Year|Rating|dense_rank|
+-------+-----+----------+----+------+----------+
|Shirley|E0589|   Finance|2020|   7.0|         1|
|Shirley|E0589|   Finance|2018|   6.0|         2|
|Shirley|E0589|   Finance|2019|   5.0|         3|
| Philip|E1027|        HR|2019|   8.5|         1|
| Philip|E1027|        HR|2020|   8.0|         2|
|  Peter|E5489| Marketing|2017|   8.5|         1|
|  Peter|E5489| Marketing|2020|   8.0|         2|
|  Peter|E5489| Marketing|2019|   7.0|         3|
|  Peter|E5489| Marketing|2016|   7.0|         3|
|  Peter|E5489| Marketing|2018|   6.5|         4|
+-------+-----+----------+----+------+----------+



In [21]:
# Filter the DataFrame to only include the second highest rating for each user
filtered_data = dense_rank.filter(col("dense_rank") == 2)
filtered_data.show()

+-------+-----+----------+----+------+----------+
|EmpName|EmpID|Department|Year|Rating|dense_rank|
+-------+-----+----------+----+------+----------+
|Shirley|E0589|   Finance|2018|   6.0|         2|
| Philip|E1027|        HR|2020|   8.0|         2|
|  Peter|E5489| Marketing|2020|   8.0|         2|
+-------+-----+----------+----+------+----------+



In [23]:
# Calculate the difference between the average rating and the second highest rating for each user

join_df = avg_rating.join(dense_rank, on="empid", how="inner")
join_df.show()

+-----+---------+-------+----------+----+------+----------+
|empid|avgRating|EmpName|Department|Year|Rating|dense_rank|
+-----+---------+-------+----------+----+------+----------+
|E0589|      6.0|Shirley|   Finance|2020|   7.0|         1|
|E0589|      6.0|Shirley|   Finance|2018|   6.0|         2|
|E0589|      6.0|Shirley|   Finance|2019|   5.0|         3|
|E1027|     8.25| Philip|        HR|2019|   8.5|         1|
|E1027|     8.25| Philip|        HR|2020|   8.0|         2|
|E5489|      7.4|  Peter| Marketing|2017|   8.5|         1|
|E5489|      7.4|  Peter| Marketing|2020|   8.0|         2|
|E5489|      7.4|  Peter| Marketing|2019|   7.0|         3|
|E5489|      7.4|  Peter| Marketing|2016|   7.0|         3|
|E5489|      7.4|  Peter| Marketing|2018|   6.5|         4|
+-----+---------+-------+----------+----+------+----------+



In [31]:
difference = join_df.withColumn("diff", round(abs(col("Rating") - col("avgRating")), 3))
difference.select("empid", "Rating", "avgRating", "diff").show()

+-----+------+---------+----+
|empid|Rating|avgRating|diff|
+-----+------+---------+----+
|E5489|   8.0|      7.4| 0.6|
|E5489|   7.0|      7.4| 0.4|
|E5489|   6.5|      7.4| 0.9|
|E5489|   8.5|      7.4| 1.1|
|E5489|   7.0|      7.4| 0.4|
|E1027|   8.0|     8.25|0.25|
|E1027|   8.5|     8.25|0.25|
|E0589|   7.0|      6.0| 1.0|
|E0589|   5.0|      6.0| 1.0|
|E0589|   6.0|      6.0| 0.0|
+-----+------+---------+----+



In [ ]:
# https://www.linkedin.com/feed/update/urn:li:activity:7115266803274055680/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115266803274055680%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

# There are two files existing_data.csv and new_data.csv for that we have following pyspark code.

# Load existing transaction data
existing_data_df = spark.read \
  .format("csv") \
  .option("header", "true") \
  .load("dbfs:/FileStore/raw/existing_data.csv")

# Load new transaction data
new_data_df = spark.read \
  .format("csv") \
  .option("header", "true") \
  .load("dbfs:/FileStore/raw/new_data.csv")

existing_data_df.show()

# Identify new records based on timestamp
max_timestamp_existing = existing_data_df.selectExpr("max(timestamp)").collect()[0][0]
new_records_df = new_data_df.filter(new_data_df.timestamp > max_timestamp_existing)

# Append new records to existing data
updated_data_df = existing_data_df.union(new_records_df)

# Show the updated dataset
updated_data_df.show()

In [32]:
# https://www.linkedin.com/feed/update/urn:li:activity:7113498071069540352/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7113498071069540352%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

schema_=["sale_id","product_name","sale_date"]

data=[
(1,"LCPHONE","2000-01-16"),
(2,"LCPhone","2000-01-17"),
(3,"LcPhOnE","2000-02-18"),
(4,"LCKeyCHAiN","2000-02-19"),
(5,"LCKeyChain","2000-02-28"),
(6,"Matryoshka","2000-03-31")]

df = spark.createDataFrame(data, schema=schema_)
df.show()

+-------+------------+----------+
|sale_id|product_name| sale_date|
+-------+------------+----------+
|      1|     LCPHONE|2000-01-16|
|      2|     LCPhone|2000-01-17|
|      3|     LcPhOnE|2000-02-18|
|      4|  LCKeyCHAiN|2000-02-19|
|      5|  LCKeyChain|2000-02-28|
|      6|  Matryoshka|2000-03-31|
+-------+------------+----------+



In [36]:
lowerData = df.withColumn("product_name", lower(trim(col("product_name"))))
lowerData.show()

+-------+------------+----------+
|sale_id|product_name| sale_date|
+-------+------------+----------+
|      1|     lcphone|2000-01-16|
|      2|     lcphone|2000-01-17|
|      3|     lcphone|2000-02-18|
|      4|  lckeychain|2000-02-19|
|      5|  lckeychain|2000-02-28|
|      6|  matryoshka|2000-03-31|
+-------+------------+----------+



In [38]:
monthData = lowerData.withColumn("sale_month", date_format(col("sale_date"), "yyyy-MM"))
monthData.show()

+-------+------------+----------+----------+
|sale_id|product_name| sale_date|sale_month|
+-------+------------+----------+----------+
|      1|     lcphone|2000-01-16|   2000-01|
|      2|     lcphone|2000-01-17|   2000-01|
|      3|     lcphone|2000-02-18|   2000-02|
|      4|  lckeychain|2000-02-19|   2000-02|
|      5|  lckeychain|2000-02-28|   2000-02|
|      6|  matryoshka|2000-03-31|   2000-03|
+-------+------------+----------+----------+



In [48]:
grData = monthData.groupBy("sale_month", "product_name")\
        .agg(count("sale_month").alias("total"))\
        .orderBy("product_name", "sale_month")
grData.show()

finalData = grData.select("product_name", "sale_month", "total")
finalData.show()

+----------+------------+-----+
|sale_month|product_name|total|
+----------+------------+-----+
|   2000-02|  lckeychain|    2|
|   2000-01|     lcphone|    2|
|   2000-02|     lcphone|    1|
|   2000-03|  matryoshka|    1|
+----------+------------+-----+

+------------+----------+-----+
|product_name|sale_month|total|
+------------+----------+-----+
|  lckeychain|   2000-02|    2|
|     lcphone|   2000-01|    2|
|     lcphone|   2000-02|    1|
|  matryoshka|   2000-03|    1|
+------------+----------+-----+



In [49]:
# https://www.linkedin.com/feed/update/urn:li:activity:7115260055872032769/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115260055872032769%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
# Write a SQL query to identify 𝒕𝒉𝒆 𝑯𝒊𝒈𝒉𝒆𝒔𝒕 & 𝑳𝒐𝒘𝒆𝒔𝒕 𝑺𝒂𝒍𝒂𝒓𝒊𝒆𝒅 𝑬𝒎𝒑𝒍𝒐𝒚𝒆𝒆 𝒊𝒏 𝒆𝒂𝒄𝒉 𝑫𝒆𝒑𝒂𝒓𝒕𝒎𝒆𝒏𝒕.

data = [(1001, 'Marlania', 92643, 1),
(1002, 'Briana', 87202, 1),
(1003, 'Maysha', 70545, 1),
(1004, 'Jamacia', 65285, 1),
(1005, 'Kimberli', 51407, 2),
(1006, 'Lakken', 88933, 2),
(1007, 'Micaila', 82145, 2),
(1008, 'Gion', 66187, 2),
(1009, 'Latoynia', 55729, 3),
(1010, 'Shaquria', 52111, 3),
(1011, 'Tarvares', 82979, 3),
(1012, 'Gabriella', 74132, 4),
(1013, 'Medusa', 72551, 4),
(1014, 'Kubra', 55170, 4)]

schema_ = ["emp_id", "emp_name" ,"salary", "dep_id"]

df = spark.createDataFrame(data, schema=schema_)
df.show()

+------+---------+------+------+
|emp_id| emp_name|salary|dep_id|
+------+---------+------+------+
|  1001| Marlania| 92643|     1|
|  1002|   Briana| 87202|     1|
|  1003|   Maysha| 70545|     1|
|  1004|  Jamacia| 65285|     1|
|  1005| Kimberli| 51407|     2|
|  1006|   Lakken| 88933|     2|
|  1007|  Micaila| 82145|     2|
|  1008|     Gion| 66187|     2|
|  1009| Latoynia| 55729|     3|
|  1010| Shaquria| 52111|     3|
|  1011| Tarvares| 82979|     3|
|  1012|Gabriella| 74132|     4|
|  1013|   Medusa| 72551|     4|
|  1014|    Kubra| 55170|     4|
+------+---------+------+------+



In [51]:
max_min = df.groupBy("dep_id").agg(max("salary").alias("maxSalary"), min("salary").alias("minSalary"))
max_min.show()

joinedData = df.withColumnRenamed("dep_id", "dept_id")
joinedData = joinedData.join(max_min, joinedData.dept_id == max_min.dep_id, "inner") 
joinedData.show()

resultDF = joinedData.groupBy("dep_id").agg(
    max(when(col("salary") == col("maxSalary"), col("emp_name")))\
    .alias("max_emp"),
    min(when(col("salary") == col("minSalary"), col("emp_name"))\
        .otherwise("null")).alias("min_emp"))
resultDF.show()

+------+---------+---------+
|dep_id|maxSalary|minSalary|
+------+---------+---------+
|     1|    92643|    65285|
|     2|    88933|    51407|
|     3|    82979|    52111|
|     4|    74132|    55170|
+------+---------+---------+

+------+---------+------+-------+------+---------+---------+
|emp_id| emp_name|salary|dept_id|dep_id|maxSalary|minSalary|
+------+---------+------+-------+------+---------+---------+
|  1001| Marlania| 92643|      1|     1|    92643|    65285|
|  1002|   Briana| 87202|      1|     1|    92643|    65285|
|  1003|   Maysha| 70545|      1|     1|    92643|    65285|
|  1004|  Jamacia| 65285|      1|     1|    92643|    65285|
|  1005| Kimberli| 51407|      2|     2|    88933|    51407|
|  1006|   Lakken| 88933|      2|     2|    88933|    51407|
|  1007|  Micaila| 82145|      2|     2|    88933|    51407|
|  1008|     Gion| 66187|      2|     2|    88933|    51407|
|  1009| Latoynia| 55729|      3|     3|    82979|    52111|
|  1010| Shaquria| 52111|      3|  

In [ ]:
# --------------------

In [1]:
# Normal Imports
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import col,struct,when, lit
from pyspark.sql import Row

print("imported")
# Agg

from pyspark.sql.functions import approx_count_distinct,collect_list
from pyspark.sql.functions import collect_set, sum, avg, max, countDistinct, count
from pyspark.sql.functions import first, last, kurtosis, min, mean, skewness 
from pyspark.sql.functions import stddev, stddev_samp, stddev_pop, sumDistinct
from pyspark.sql.functions import variance, var_samp, var_pop

from pyspark.sql.functions import col,avg,sum,min,max,row_number

from pyspark.sql.functions import *
from pyspark.sql.window import Window

from pyspark.sql.types import MapType, StructType, StructField, StringType, IntegerType
from pyspark.sql.types import *
from pyspark.sql.functions import from_json

print("imported")
# spark context
conf = SparkConf().setAppName("Spark").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf = conf)
sc.setLogLevel("ERROR")


# spark session
spark = SparkSession.builder.master("local[1]").appName('SparkByExamples.com').getOrCreate()

# Usage of config()
# spark = SparkSession.builder \
#       .master("local[1]") \
#       .appName("LearnSpark") \
#       .config("spark.some.config.option", "config-value") \
#       .getOrCreate()

print("imported All")


imported
imported


23/12/29 19:56:34 WARN Utils: Your hostname, DINESHs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.100 instead (on interface en0)
23/12/29 19:56:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/29 19:56:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


imported All


In [50]:
# https://www.linkedin.com/posts/mahaboob-pathan_snowflake-snowsql-sql-activity-7132574458640228352-AqzH

# see all the previous posta nd do in sql



In [2]:
# https://www.linkedin.com/posts/mahaboob-pathan_namastesql-sqlquery-sqlchallenges-activity-7113771061086744576-xfnO

schema_ = ["id", "student"]

data = [(1, 'A'),
(2, 'D'),
(3, 'E'),
(4, 'G'),
(5, 'J')]

df = spark.createDataFrame(data, schema= schema_)
df.show()

+---+-------+
| id|student|
+---+-------+
|  1|      A|
|  2|      D|
|  3|      E|
|  4|      G|
|  5|      J|
+---+-------+



In [5]:
lead_fun = Window.orderBy(asc("id"))

lead_lag = df.withColumn("lead", lead("student").over(lead_fun))\
        .withColumn("lag", lag("student").over(lead_fun))
lead_lag.show()

+---+-------+----+----+
| id|student|lead| lag|
+---+-------+----+----+
|  1|      A|   D|null|
|  2|      D|   E|   A|
|  3|      E|   G|   D|
|  4|      G|   J|   E|
|  5|      J|null|   G|
+---+-------+----+----+



In [11]:
lead_lag.withColumn("res", 
        when(col("id")%2 != 0, when(col('lead') == "null", 
                col("student")).otherwise(col("lead")))\
                    .when(col('id') % 2 == 0, col("lag"))
                   ).show()

+---+-------+----+----+----+
| id|student|lead| lag| res|
+---+-------+----+----+----+
|  1|      A|   D|null|   D|
|  2|      D|   E|   A|   A|
|  3|      E|   G|   D|   G|
|  4|      G|   J|   E|   E|
|  5|      J|null|   G|null|
+---+-------+----+----+----+



In [17]:
df.createOrReplaceTempView("seats_tbl")
spark.sql("select \
            case when mod(ID,2) = 0 then ID-1 \
                  when mod(ID,2) <> 0 and ID <> (select Max(ID) \
                from seats_tbl) then ID+1 else ID end as ID, \
                  STUDENT from seats_tbl order by 1;"
         ).show()

+---+-------+
| ID|STUDENT|
+---+-------+
|  1|      D|
|  2|      A|
|  3|      G|
|  4|      E|
|  5|      J|
+---+-------+



In [ ]:
# https://www.linkedin.com/posts/mahaboob-pathan_namastesql-sqlquery-sqlchallenges-activity-7113004314930933760-gXk8

# Write a query to get 𝒕𝒉𝒆 𝒏𝒐 𝒐𝒇 𝒖𝒏𝒊𝒒𝒖𝒆 𝒏𝒂𝒎𝒆𝒔, 𝒏𝒐 𝒐𝒇 𝒕𝒓𝒂𝒏𝒔𝒂𝒄𝒕𝒊𝒐𝒏𝒔, 𝒂𝒏𝒅 𝒕𝒉𝒆 𝒅𝒊𝒇𝒇𝒆𝒓𝒆𝒏𝒄𝒆 𝒃𝒆𝒕𝒘𝒆𝒆𝒏 𝒕𝒉𝒆 𝒇𝒊𝒓𝒔𝒕 & 𝒍𝒂𝒔𝒕 𝒕𝒓𝒂𝒏𝒔𝒂𝒄𝒕𝒊𝒐𝒏 𝒐𝒄𝒄𝒖𝒓𝒆𝒅 𝒐𝒏 02-01-2023.

# https://www.linkedin.com/feed/update/urn:li:activity:7115260055872032769/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115260055872032769%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29




In [18]:
https://www.linkedin.com/feed/update/urn:li:activity:7114235881678417920/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7114235881678417920%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

SyntaxError: invalid decimal literal (3209194749.py, line 1)

In [20]:
https://www.linkedin.com/feed/update/urn:li:activity:7114530651709554688/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7114530651709554688%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

SyntaxError: invalid decimal literal (1727679684.py, line 1)

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7110481458405785600/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7110481458405785600%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29


𝐇𝐨𝐰 𝐒𝐩𝐚𝐫𝐤 𝐝𝐞𝐜𝐢𝐝𝐞𝐬 𝐨𝐧 𝐭𝐡𝐞 𝐉𝐨𝐢𝐧 𝐒𝐭𝐫𝐚𝐭𝐞𝐠𝐲?

Spark uses a cost-based optimizer to choose the most efficient join strategy based on the size of the datasets, the type of join, the join condition, the distribution of the data, and the availability of resources.

The three most common join strategies in Spark are:
1. Broadcast join
2. Shuffle hash join
3. Sort merge join

Read my blog to learn more about how Apache Spark decides on the join strategy and how to use join hints effectively: https://lnkd.in/g3EvZZfC

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7110903214325059585/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7110903214325059585%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7114818781520048128/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7114818781520048128%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
Resources to Prepare:
1. DataLemur (Really good questions from product based companies but edge cases not covered). [https://datalemur.com/]
2. LeetCode (the classic). [https://leetcode.com/]
3. 8 week SQL challenge. [https://lnkd.in/g9KxBFem]
4. This cult classic problem:
Table A: 1,1,2,Null,3,5,Null
Table B: 1,2,2,Null,4,5,Null,6
What will be the result of Inner, Left, Right, Full, Cross Join?

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7115546144084037633/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115546144084037633%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7115266803274055680/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115266803274055680%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7115588607486156801/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115588607486156801%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7115644296111783936/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115644296111783936%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7115697833059491840/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115697833059491840%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7115373323517145088/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115373323517145088%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7115546153542172672/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115546153542172672%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7116437104812851200/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7116437104812851200%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7116106222910550016/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7116106222910550016%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7116006679380443136/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7116006679380443136%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7115897323930460160/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7115897323930460160%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7117297691230957568/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7117297691230957568%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7117038244063510528/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7117038244063510528%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7116973047328206848/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7116973047328206848%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7116830873060069376/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7116830873060069376%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7116565359666610176/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7116565359666610176%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7116608449236353024/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7116608449236353024%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7116565359335268352/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7116565359335268352%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
https://github.com/imadhaka/PyLeetProblems/blob/master/spark/PairedRows.py
    
https://github.com/imadhaka/PyLeetProblems/tree/master

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7116437104812851200/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7116437104812851200%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7116445887807074304/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7116445887807074304%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7117350533186695168/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7117350533186695168%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7117377810200952832/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7117377810200952832%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7117103379687821313/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7117103379687821313%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7117667622749421568/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7117667622749421568%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7118105404118593538/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7118105404118593538%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7118037578146070528/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7118037578146070528%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7118305823767883776/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7118305823767883776%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7117841840699052032/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7117841840699052032%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7118075350764445697/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7118075350764445697%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7118772247946797056/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7118772247946797056%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7118472512916643840/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7118472512916643840%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7118426384892379136/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7118426384892379136%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7118905100168777729/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7118905100168777729%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7119913279933603840/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7119913279933603840%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7119548123739226112/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7119548123739226112%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7119597076719509504/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7119597076719509504%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7119912119998840832/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7119912119998840832%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7119585361919418368/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7119585361919418368%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7120995100997939200/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7120995100997939200%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
📌Question: Given one list of events like below example.
list1 = ['user1 click 06:04:00', 'user2 log ERROR', 'user1 click 04:09:08', 'user1 click 00:00:01']

🎯show me the distinct number of users.
🎯show me the how many number of events.
🎯show me the which users having most events.

🍀For the above tasks, we can easly write the code in normal programming, but we need to write in the RDD form.                
                
                

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7120651975347597312/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7120651975347597312%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7120618783588687872/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7120618783588687872%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7120110212866306048/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7120110212866306048%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7120256705694736385/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7120256705694736385%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7121112905302863873/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7121112905302863873%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7118496721755648000/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7118496721755648000%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7121905553978593280/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7121905553978593280%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7122282162531241984/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7122282162531241984%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7122199508280606720/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7122199508280606720%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7122128627739086848/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7122128627739086848%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7122001177667395585/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7122001177667395585%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
https://github.com/imadhaka/PyLeetProblems/blob/master/spark/StudentAttendance.py                

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7121832036981501952/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7121832036981501952%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29


https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/4602256280705183/69616450320427/8093088653480119/latest.html

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7120677975829753856/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7120677975829753856%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7121343994546053120/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7121343994546053120%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7120678141882179585/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7120678141882179585%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7122827934645563393/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7122827934645563393%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7122634055266803712/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7122634055266803712%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
do all the questins in this.                

In [ ]:
https://github.com/mattmartin14/dream_machine/tree/main/deltalake/docker/dl1

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7122938682243772416/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7122938682243772416%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7121361196137484288/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7121361196137484288%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7123234638797373440/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7123234638797373440%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7123345666528935936/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7123345666528935936%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7123353474204852224/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7123353474204852224%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
Sample data:
product_id,product_name,quantity_sold,order_date
1,Product A,50,2023-07-01
2,Product B,75,2023-07-05
3,Product C,30,2023-07-12
4,Product A,45,2023-07-15
5,Product D,60,2023-07-20
6,Product B,70,2023-07-25
7,Product C,35,2023-07-29
8,Product D,55,2023-07-30
9,Product E,40,2023-07-01
10,Product F,65,2023-07-10
              

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7120956804745826304/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7120956804745826304%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7123284641020579843/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7123284641020579843%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7123527963072970752/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7123527963072970752%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7123522447164776448/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7123522447164776448%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7124085577792073728/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7124085577792073728%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7123558824749236224/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7123558824749236224%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7124256858676035584/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7124256858676035584%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7123896977603387392/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7123896977603387392%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7124450661517131776/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7124450661517131776%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7124446283817201664/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7124446283817201664%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7125009569696673794/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7125009569696673794%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7125014408900083713/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7125014408900083713%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
watch all the questions                

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7125185637636333568/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7125185637636333568%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7126894271420387328/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7126894271420387328%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127097269199245312/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127097269199245312%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7126623558360780800/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7126623558360780800%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7126445296313593856/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7126445296313593856%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7126262755782799360/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7126262755782799360%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7125706674941607936/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7125706674941607936%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7125538528997892096/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7125538528997892096%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127574447221260288/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127574447221260288%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127284282821070848/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127284282821070848%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127518232948654080/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127518232948654080%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127187913859104768/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127187913859104768%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127270902743896064/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127270902743896064%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127500819242876928/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127500819242876928%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127343624580739072/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127343624580739072%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127648844800008193/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127648844800008193%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127242879789330432/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127242879789330432%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127575459453648896/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127575459453648896%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127656206487142400/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127656206487142400%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7127867344248291328/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7127867344248291328%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7128353648228675584/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7128353648228675584%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7128290390381592576/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7128290390381592576%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7128161770971496448/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7128161770971496448%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7130131940963143681/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7130131940963143681%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29#

In [ ]:
Okay! As a Data Engineer, how often would you have written the below piece of code?

Spark = SparkSession.builder
.master()
.appName()
.config()
.getOrCreate()

I would say you have used it more often. But ever wondered why we write this?

✅ A SparkSession is the entry point to programming Spark with the DataFrame and SQL API. It is the unified interface that combines the functionalities of SQLContext, HiveContext, and StreamingContext

✅ builder is an attribute within SparkSession class which initiates Builder class to construct :class:`SparkSession` instances. The Builder class contains methods that allow you to set various configuration options for the SparkSession before creating it.

Below are some of the methods within Builder class,

❎ master(String master) :- Used to specify the cluster manager URL, determining where the Spark application will run. You can set this value to connect to a standalone cluster, YARN, Mesos, or Kubernetes, depending on your infrastructure

❎ appName(String name) :- Sets a name for the application, which will be shown in the Spark web UI. If no application name is set, a randomly generated name will be used.

❎ config(SparkConf conf) :- Used to set configuration options for your Spark application. It allows you to specify various Spark properties and control the behavior of your Spark job.

❎ getOrCreate() :- Used to either retrieve an existing SparkSession with the specified configuration or create a new one if it doesn't exist.

Edited:- GetorCreate ensure instance of Spark session remains one per spark application. In Apache Spark, it is possible to create and use multiple Spark sessions within a single application. Multiple Spark sessions within the same application are typically used in scenarios where you want to isolate different sets of Spark functionality or configurations.

❗ Note:- While it's possible to have multiple Spark sessions in a single application, doing so comes with certain trade-offs and potential drawbacks. Each Spark session consumes resources, including memory and CPU. Creating multiple sessions within the same application can lead to increased resource overhead!

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7129973716570017792/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7129973716570017792%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7129811895481475072/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7129811895481475072%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7130188087300636672/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7130188087300636672%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7130094886023761920/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7130094886023761920%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7130590802245779458/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7130590802245779458%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7130053191857045504/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7130053191857045504%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

1) How do You execute your Spark Job ?
Spark-submit
It has many parameters such as -
>>spark-submit \
- - master yarn \       resource manager is Yarn
- - deploy-mode cluster \     deploying the spark application in cluster mode
- - driver-memory 8g \         memory to be used by driver jvm
- - driver-cores 1 \    cores to be used by driver jvm
- - num-executors 5\ The total number of executors to be used
- - executor-memory 16g \  Amount of memory to be used ach executors
- - executor-cores 2 \ Number of cores to be used by each executors
- - conf "spark.sql.shuffle.partitions=20000" \       
- - conf "spark.executor.memoryOverhead=5244" \
- - conf "spark.memory.fraction=0.8" \
- - conf "spark.memory.storageFraction=0.2" \
- - conf "spark.serializer=org.apache.spark.serializer.KryoSerializer" \
- - conf "spark.sql.files.maxPartitionBytes=168435456" \
- - conf "spark.dynamicAllocation.minExecutors=1" \
- - conf "spark.dynamicAllocation.maxExecutors=200" \
- - conf "spark.dynamicAllocation.enabled=true" \
- - conf "spark.executor.extraJavaOptions=-XX:+PrintGCDetails -XX:+PrintGCTimeStamps" \ 
- - jars dependency1.jar, dependency2.jar \
- - class 
 

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7130220537133301760/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7130220537133301760%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7130826889874604034/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7130826889874604034%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7131430811504963585/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7131430811504963585%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7131150748666523648/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7131150748666523648%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [21]:
input_sentence = "This is a sample sentence to reverse"
print(input_sentence[::-1])

esrever ot ecnetnes elpmas a si sihT


In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7130904493835120640/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7130904493835120640%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7131849051863400448/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7131849051863400448%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7131286673409114112/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7131286673409114112%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7131589365658517504/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7131589365658517504%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7131308221222293505/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7131308221222293505%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7132352789942255617/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7132352789942255617%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7132377874984022016/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7132377874984022016%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7132233433853886465/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7132233433853886465%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7131250476611768321/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7131250476611768321%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7132210496119734273/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7132210496119734273%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7131800751231696897/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7131800751231696897%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/posts/priyam-jain-0946ab199_pyspark-pyspark-spark-activity-7131308221222293505-Kt1i/?utm_source=share&utm_medium=member_desktop

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7131677453747429376/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7131677453747429376%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7133439078947037184/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7133439078947037184%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7133300632303587328/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7133300632303587328%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7133650440394616832/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7133650440394616832%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7133038914390392835/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7133038914390392835%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7132887915352784896/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7132887915352784896%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7132784170765885440/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7132784170765885440%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7132872813538910208/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7132872813538910208%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7133612690916184064/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7133612690916184064%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7133257864789327873/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7133257864789327873%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7133272955505033218/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7133272955505033218%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/posts/priyam-jain-0946ab199_pyspark-pyspark-spark-activity-7132377874984022016-m2kx/?utm_source=share&utm_medium=member_desktop

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7133125938602491904/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7133125938602491904%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7135466730939682816/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7135466730939682816%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
𝐒𝐡𝐚𝐫𝐞𝐝 𝐕𝐚𝐫𝐢𝐚𝐛𝐥𝐞𝐬 𝐢𝐧 𝐒𝐩𝐚𝐫𝐤
=================================
There are 2 type of shared variables
i)Broadcast variable
ii) Accumulator variable

𝐁𝐫𝐨𝐚𝐝𝐜𝐚𝐬𝐭 𝐕𝐚𝐫𝐢𝐚𝐛𝐥𝐞 :
-> Broadcast variables are read-only, distributed data structures used to efficiently share data across worker nodes in a Spark cluster
-> Broadcast Variables are used to cache a value or object on each node in the cluster, which can then be used across multiple tasks.
-> They are loaded onto every worker node so that they can access them.

 𝐔𝐬𝐞 𝐜𝐚𝐬𝐞 : You can use broadcast to distribute lookup tables or reference data to each worker node, used in a join operation or filter operation.
Ex : val joinedDf=largeDf.join(broadcast(smallDf))
=======================================
𝐀𝐜𝐜𝐮𝐦𝐮𝐥𝐚𝐭𝐨𝐫 :
->Accumulator variable is a shared copy kept in driver node and all the executors will have access to it.
->Executors update the variable in Driver node.
->Executors only have an access to update the variable they do not have access to read the variable.

𝐔𝐬𝐞 𝐜𝐚𝐬𝐞 :
you can use an accumulator to keep track of the count of a particular event, sum of values, or any other kind of accumulation that you need to do across all the worker nodes
Ex : val accum = sc.longAccumulator("Sum Accumulator")
val rdd=sc.parallelize(Array(1, 2, 3)).foreach(x => accum.add(x))
println(accum.value)                

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7130236033287274497/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7130236033287274497%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7134163826815418368/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7134163826815418368%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7133831650236411904/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7133831650236411904%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/posts/priyam-jain-0946ab199_databricks-pyspark-spark-activity-7134594743274995712-4LwC/?utm_source=share&utm_medium=member_desktop

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7134960321202446336/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7134960321202446336%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7134964193274843139/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7134964193274843139%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7134766136801263617/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7134766136801263617%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7134424091536572416/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7134424091536572416%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7133878542810951680/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7133878542810951680%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7136989323773493248/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7136989323773493248%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7133310348861861888/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7133310348861861888%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7135618539335389185/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7135618539335389185%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7135657443992248321/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7135657443992248321%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7136950876237144064/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7136950876237144064%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7136722175230689280/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7136722175230689280%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
project: https://github.com/Pavanpawar2705/Ingesting-Real-time-Logistics-Data-in-MongoDB-with-Kafka-and-Python                

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7136421513045434368/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7136421513045434368%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7137026016887140352/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7137026016887140352%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7135663369507803136/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7135663369507803136%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7138387159870296064/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7138387159870296064%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7135475274451664896/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7135475274451664896%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7137478380425285632/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7137478380425285632%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7138499356214665218/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7138499356214665218%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
https://www.montecarlodata.com/blog-pyspark-data-quality-checks

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7138583044248035328/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7138583044248035328%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7141326742601863168/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7141326742601863168%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
Deloitte Data Engineer Interview question and Round :
soultuion:
Q:- How would you rename 100 column in pyspark?
Ans:-
from functools import reduce
def rename_cols(df, old_columns, new_columns):
  for old_col, new_col in zip(old_columns, new_columns):
    df = df.withColumnRenamed(old_col, new_col)
  return df

old_columns = ['old_name1', 'old_name2', ..., 'old_name100']
new_columns = ['new_name1', 'new_name2', ..., 'new_name100']

df_renamed = rename_cols(df, old_columns, new_columns)                

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7140921123524837376/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7140921123524837376%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

🛑 Features of PySpark :
===================

1. In memory computation
2. Distributed processing using parallelism
3. Fault - tolerance
4. Immutable
5. Lazy Evaluation
6. Cache & Persistence
7. Inbuild optimization using DataFrame

🛑 Advantage of PySpark
==================

1. It is a general purpose in-memory distributed processing engine that allows to process data efficiently in a distributed manner.

2. Application running on spark is 100x faster than traditional one (i.e Hadoop ecosystem)

3. We can process data from external data resource like HDFS, AWS, S3 & many more file system.

4. It has Mlib & graph libraries as well for processing.

5. By spark we can process real-time data through streaming & Kafka.


In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7140407880188428288/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7140407880188428288%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
Data Engineering Interview Question:
===========================

Spark map() vs mapPartitions():
========================

map() –

Spark map() transformation applies a function to each row in a DataFrame/Dataset and returns the new transformed Dataset.


mapPartitions() –

This is exactly the same as map(); the difference being, Spark mapPartitions() provides a facility to do heavy initializations (for example Database connection) once for each partition instead of doing it on every DataFrame row. This helps the performance of the job when you dealing with heavy-weighted initialization on larger datasets.

map vs flatmap :
=============

The key difference between map and flatMap is map returns only one row/element for every input, while flatMap() can return a list of rows/elements.                
                

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7140257445376937984/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7140257445376937984%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29


Data Engineering Interview question:
===========================

distinct() vs dropDuplicates() :


distinct() and dropDuplicates() in Spark are used to remove duplicate rows, but there is a subtle difference. distinct() considers all columns when identifying duplicates, while dropDuplicates() allowing you to specify a subset of columns to determine uniqueness.

Please comment on which one is better if we consider we need to remove duplicate rows.


In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7140395946852429824/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7140395946852429824%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7141653171134787584/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7141653171134787584%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7143547646807355393/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7143547646807355393%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7142294899235696640/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7142294899235696640%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7143112135022133249/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7143112135022133249%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7143210451340804097/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7143210451340804097%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7143268521010008067/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7143268521010008067%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7144726008561041408/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7144726008561041408%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7143508594397724672/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7143508594397724672%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7144663579512791040/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7144663579512791040%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7143656501138673667/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7143656501138673667%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7144167235375280128/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7144167235375280128%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7142271906786725888/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7142271906786725888%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7144922987287240704/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7144922987287240704%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7143818058900324352/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7143818058900324352%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                

see the pdf and see                

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7145061670602678272/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7145061670602678272%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
This is the best explanation to differentiate🤯.. Let’s learn…

1️⃣ 𝗗𝗮𝘁𝗮 𝗪𝗮𝗿𝗲𝗵𝗼𝘂𝘀𝗲: A data warehouse is a centralized repository of data that is used for analysis and reporting. It is typically used to store historical data, but can also store real-time data. Data warehouses are typically structured, meaning that the data is organized in a way that makes it easy to query and analyze.🏢

2️⃣ 𝗗𝗮𝘁𝗮 𝗠𝗮𝗿𝘁: A data mart is a subset of a data warehouse. It's designed to cater to the needs of a specific business unit or team. Think of it as a department store within the larger shopping mall (the data warehouse).🏪

3️⃣ 𝗗𝗮𝘁𝗮 𝗟𝗮𝗸𝗲: A data lake is a repository for all of an organization's data, both structured and unstructured. It is a more flexible and scalable solution than a data warehouse, as it can store any type of data, regardless of its structure. Data lakes are typically used for big data analytics, as they can store and process large amounts of data quickly and efficiently..🌊

4️⃣ 𝗗𝗲𝗹𝘁𝗮 𝗟𝗮𝗸𝗲: Delta Lake is a data lake storage layer that provides a unified view of data across multiple data lakes. It is a relatively new technology, but it has quickly become popular due to its ability to provide a consistent view of data in a data lake.

• ACID transactions
• Data versioning
• Automatic compaction🛠️

5️⃣ 𝗗𝗮𝘁𝗮 𝗣𝗶𝗽𝗲𝗹𝗶𝗻𝗲: A data pipeline is the process of moving data from one location to another, often involving steps for extracting, transforming, and loading data (ETL). It's like the conveyer belt in a factory, ensuring that the data gets to where it's needed in a usable form.🚀

6️⃣ 𝗗𝗮𝘁𝗮 𝗠𝗲𝘀𝗵: Data Mesh is an architectural paradigm that treats data as a product. It decentralizes data ownership and architecture, allowing teams to develop, govern, and operate their own data domains. It's a shift from monolithic, centralized data management to a more distributed approach.🕸️

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7144217757201358849/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7144217757201358849%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7146341572584816640/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7146341572584816640%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7146286978206949376/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7146286978206949376%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7146478784010006529/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7146478784010006529%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
# -------------------------
others

In [ ]:
https://www.linkedin.com/posts/priyam-jain-0946ab199_pyspark-pyspark-databricks-activity-7116845920289234944-9Zvn/?utm_source=share&utm_medium=member_desktop
    
links to previous questions    

do all the links

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7118563608191533056/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7118563608191533056%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
do in last        

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7120828695124074497/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7120828695124074497%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/posts/priyam-jain-0946ab199_pyspark-pyspark-spark-activity-7126262755782799360-zsjS/?utm_source=share&utm_medium=member_desktop

In [ ]:
40

https://www.linkedin.com/feed/update/urn:li:activity:7131308221222293505/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7131308221222293505%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7131113063897800705/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7131113063897800705%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7130992927283052544/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7130992927283052544%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7136270124465999872/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7136270124465999872%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
try all this                

In [ ]:
project

https://github.com/MrSachinGoyal/gcp-dataproc-pyspark-analytics

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7145633755364229120/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7145633755364229120%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29

In [ ]:
https://www.linkedin.com/feed/update/urn:li:activity:7146764370142724097/?updateEntityUrn=urn%3Ali%3Afs_updateV2%3A%28urn%3Ali%3Aactivity%3A7146764370142724097%2CFEED_DETAIL%2CEMPTY%2CDEFAULT%2Cfalse%29
                
5 SQL projects to build your strong Analytics portfolio 🚀
                